# DEAM Dataset - Preprocessing

# Done on local computer

## Import relevant libraries

In [1]:
import numpy as np
import ast
import pandas as pd
import matplotlib.pyplot as plt

import os

Set file directory variables

In [2]:
import sys
sys.path.insert(1, '../../utils')
from paths import *

## Verify file count

Check that the number of .csv files in the './data/DEAM/features' directory matches the number number of .mp3 files in the './data/DEAM/MEMD_audio' directory

In [3]:
def extract_filenames(directory, file_format):
    raw_file_list = os.listdir(directory)
    raw_count = len(raw_file_list)
    print(f"Total files: {raw_count} \nFiles: {raw_file_list}\n")
    print("Processing files...")
    processed_list = []
    for filename in raw_file_list:
        if file_format in filename:
            filename = filename.replace(f".{file_format}", "")
            processed_list.append(filename)
    
    processed_count = len(processed_list)
    print(f"Total files: {processed_count} \nFiles: {processed_list}")
    return processed_list, processed_count

In [4]:
features_list, features_count = extract_filenames(get_deam_path('features'), "csv")

Total files: 1802 
Files: ['223.csv', '237.csv', '551.csv', '579.csv', '1953.csv', '1947.csv', '586.csv', '1012.csv', '1774.csv', '1760.csv', '1006.csv', '592.csv', '1990.csv', '1748.csv', '1984.csv', '747.csv', '1589.csv', '1576.csv', '1210.csv', '784.csv', '790.csv', '1204.csv', '1562.csv', '2043.csv', '960.csv', '974.csv', '1238.csv', '2057.csv', '155.csv', '169.csv', '83.csv', '1402.csv', '196.csv', '68.csv', '1364.csv', '1370.csv', '182.csv', '1416.csv', '54.csv', '1358.csv', '814.csv', '40.csv', '800.csv', '431.csv', '357.csv', '343.csv', '425.csv', '1199.csv', '1827.csv', '1833.csv', '419.csv', '1166.csv', '1600.csv', '1614.csv', '380.csv', '1172.csv', '1628.csv', '1629.csv', '381.csv', '1615.csv', '1173.csv', '1167.csv', '1601.csv', '395.csv', '1832.csv', '1198.csv', '1826.csv', '342.csv', '430.csv', '356.csv', '801.csv', '41.csv', '7.csv', '815.csv', '1359.csv', '55.csv', '1371.csv', '1417.csv', '197.csv', '1403.csv', '829.csv', '1365.csv', '69.csv', '82.csv', '168.csv', '96.c

In [5]:
audio_list, audio_count = extract_filenames(get_deam_path('MEMD_audio'), "mp3")

Total files: 1802 
Files: ['746.mp3', '1588.mp3', '1563.mp3', '1205.mp3', '791.mp3', '949.mp3', '1211.mp3', '1577.mp3', '2056.mp3', '1239.mp3', '961.mp3', '2042.mp3', '550.mp3', '236.mp3', '222.mp3', '544.mp3', '1946.mp3', '1952.mp3', '1007.mp3', '1761.mp3', '1775.mp3', '1013.mp3', '1985.mp3', '1749.mp3', '1991.mp3', '342.mp3', '356.mp3', '430.mp3', '1832.mp3', '1826.mp3', '1198.mp3', '1173.mp3', '381.mp3', '1615.mp3', '1601.mp3', '395.mp3', '1167.mp3', '1629.mp3', '140.mp3', '632.mp3', '154.mp3', '82.mp3', '96.mp3', '168.mp3', '1417.mp3', '1371.mp3', '829.mp3', '1365.mp3', '69.mp3', '197.mp3', '1403.mp3', '801.mp3', '41.mp3', '815.mp3', '1359.mp3', '55.mp3', '7.mp3', '54.mp3', '1358.mp3', '814.mp3', '40.mp3', '800.mp3', '68.mp3', '1364.mp3', '1402.mp3', '196.mp3', '182.mp3', '1416.mp3', '1370.mp3', '169.mp3', '83.mp3', '155.mp3', '1628.mp3', '1600.mp3', '1166.mp3', '1172.mp3', '1614.mp3', '380.mp3', '1827.mp3', '1199.mp3', '419.mp3', '1833.mp3', '357.mp3', '431.mp3', '425.mp3', '343.m

In [6]:
if set(features_list) == set(audio_list):
    print("Features and Audio Files match!")
else:
    diff = set(features_list) - set(audio_list)
    print("Elements absent in audio:", diff)

Features and Audio Files match!


## Process the annotation dataframes

Read static song-level annotations with song_id from 1 to 2000 (2013 and 2014 only)

In [7]:
df_annotations = pd.read_csv(get_deam_path('annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv'))
display(df_annotations)
print(df_annotations.shape)

,song_id,valence_mean,valence_std,arousal_mean,arousal_std
0,2,3.1,0.94,3.0,0.63
1,3,3.5,1.75,3.3,1.62
2,4,5.7,1.42,5.5,1.63
3,5,4.4,2.01,5.3,1.85
4,7,5.8,1.47,6.4,1.69
...,...,...,...,...,...
1739,1996,3.9,1.87,5.9,2.21
1740,1997,5.3,1.42,3.9,1.14
1741,1998,6.4,1.50,6.2,1.60
1742,1999,4.6,2.11,5.4,1.80


(1744, 5)


Map the valence and arousal values in the dataset, ranging from 1 to 9, to values ranging from -1 to 1, to follow convention

In [8]:
def map_va_value(value):
  old_min = 1
  old_max = 9

  new_min = -1
  new_max = 1

  mapped_value = ((value - old_min) * (new_max - new_min) / (old_max - old_min)) + new_min
  return mapped_value

# Test the function
for i in range(1, 10):
    print(f"Original value: {i}, Mapped value: {map_va_value(i)}")

Original value: 1, Mapped value: -1.0
Original value: 2, Mapped value: -0.75
Original value: 3, Mapped value: -0.5
Original value: 4, Mapped value: -0.25
Original value: 5, Mapped value: 0.0
Original value: 6, Mapped value: 0.25
Original value: 7, Mapped value: 0.5
Original value: 8, Mapped value: 0.75
Original value: 9, Mapped value: 1.0


In [9]:
df_annotations['valence_mean_mapped'] = df_annotations[' valence_mean'].apply(map_va_value)
df_annotations['arousal_mean_mapped'] = df_annotations[' arousal_mean'].apply(map_va_value)
df_annotations = df_annotations.drop([' valence_mean', ' valence_std', ' arousal_mean', ' arousal_std'], axis=1)

In [10]:
df_annotations

,song_id,valence_mean_mapped,arousal_mean_mapped
0,2,-0.475,-0.500
1,3,-0.375,-0.425
2,4,0.175,0.125
3,5,-0.150,0.075
4,7,0.200,0.350
...,...,...,...
1739,1996,-0.275,0.225
1740,1997,0.075,-0.275
1741,1998,0.350,0.300
1742,1999,-0.100,0.100


Export the dataframe

In [11]:
df_annotations.to_csv(get_deam_path('processed/annotations/deam_static_annotations.csv'), index=False)

Import the dataframe

In [12]:
df_annotations = pd.read_csv(get_deam_path('processed/annotations/deam_static_annotations.csv'))
df_annotations

,song_id,valence_mean_mapped,arousal_mean_mapped
0,2,-0.475,-0.500
1,3,-0.375,-0.425
2,4,0.175,0.125
3,5,-0.150,0.075
4,7,0.200,0.350
...,...,...,...
1739,1996,-0.275,0.225
1740,1997,0.075,-0.275
1741,1998,0.350,0.300
1742,1999,-0.100,0.100


## Process the Essentia features datasets

Define function to check for any non-float/int columns and deal with them

In [13]:
df_essentia_features = pd.read_csv(get_deam_path('processed/features/essentia_features.csv'))
df_essentia_features

,Unnamed: 0,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,...,metadata.version.essentia_git_sha,metadata.version.extractor,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,D,minor,D,minor,A,minor
1,1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,E,minor,A,minor,A,minor,A,minor
2,2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,D,minor,D,minor,A,minor
3,3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,B,major,B,major,B,major,B,major
4,4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,G,major,C,major,C,major,C,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,E,minor,E,minor,E,minor,E,minor
1740,1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,F,major,F,major,C,major
1741,1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,F#,minor,F#,major,F#,major,F#,major
1742,1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,major,A,major,A,minor,Ab,minor


Drop the 'Unnamed: 0	' column

In [14]:
# drop Unnamed:0 column
df_essentia_features = df_essentia_features[df_essentia_features.columns[1:]]
df_essentia_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,metadata.version.essentia_git_sha,metadata.version.extractor,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,D,minor,D,minor,A,minor
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,E,minor,A,minor,A,minor,A,minor
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,D,minor,D,minor,A,minor
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,B,major,B,major,B,major,B,major
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,G,major,C,major,C,major,C,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,E,minor,E,minor,E,minor,E,minor
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,F,major,F,major,C,major
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,F#,minor,F#,major,F#,major,F#,major
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,major,A,major,A,minor,Ab,minor


See what features are available

In [15]:
print(df_essentia_features.columns.to_list())

['song_id', 'lowlevel.average_loudness', 'lowlevel.barkbands_crest.dmean', 'lowlevel.barkbands_crest.dmean2', 'lowlevel.barkbands_crest.dvar', 'lowlevel.barkbands_crest.dvar2', 'lowlevel.barkbands_crest.max', 'lowlevel.barkbands_crest.mean', 'lowlevel.barkbands_crest.median', 'lowlevel.barkbands_crest.min', 'lowlevel.barkbands_crest.stdev', 'lowlevel.barkbands_crest.var', 'lowlevel.barkbands_flatness_db.dmean', 'lowlevel.barkbands_flatness_db.dmean2', 'lowlevel.barkbands_flatness_db.dvar', 'lowlevel.barkbands_flatness_db.dvar2', 'lowlevel.barkbands_flatness_db.max', 'lowlevel.barkbands_flatness_db.mean', 'lowlevel.barkbands_flatness_db.median', 'lowlevel.barkbands_flatness_db.min', 'lowlevel.barkbands_flatness_db.stdev', 'lowlevel.barkbands_flatness_db.var', 'lowlevel.barkbands_kurtosis.dmean', 'lowlevel.barkbands_kurtosis.dmean2', 'lowlevel.barkbands_kurtosis.dvar', 'lowlevel.barkbands_kurtosis.dvar2', 'lowlevel.barkbands_kurtosis.max', 'lowlevel.barkbands_kurtosis.mean', 'lowlevel.ba

Get song_ids

In [16]:
song_ids = df_essentia_features['song_id'].values.tolist()
print(song_ids)

[2, 3, 4, 5, 7, 8, 10, 12, 13, 17, 18, 19, 20, 21, 22, 24, 25, 31, 32, 35, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 62, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 140, 142, 143, 144, 145, 146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 250, 

Some features are irrelevant, or are metadata. Drop them

In [17]:
metadata_columns = [col for col in df_essentia_features.columns if 'metadata' in col]
df_essentia_features = df_essentia_features.drop(columns=metadata_columns)
df_essentia_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,lowlevel.mfcc.cov,lowlevel.mfcc.icov,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,[[ 3.01424023e+03 -4.94424295e+00 -3.55986908e...,[[ 4.87567624e-04 2.20520218e-04 3.65056156e...,A,minor,D,minor,D,minor,A,minor
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,[[ 3.86454907e+03 -1.79961096e+03 7.49169739e...,[[ 6.12916250e-04 5.59243083e-04 -4.18933138e...,E,minor,A,minor,A,minor,A,minor
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,[[ 5.14496680e+03 -8.42719910e+02 2.21168961e...,[[ 2.5328118e-04 1.2687490e-04 -6.1599618e-05...,A,minor,D,minor,D,minor,A,minor
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,[[ 4.51047266e+03 -1.35743689e+03 -4.70529022e...,[[ 4.80499206e-04 4.93057130e-04 4.33660607e...,B,major,B,major,B,major,B,major
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,[[ 4.63236475e+03 -1.50898779e+03 -2.37786453e...,[[ 4.38551302e-04 4.59792936e-04 4.48488398e...,G,major,C,major,C,major,C,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,[[ 1.84772595e+03 -1.61741650e+03 5.11341644e...,[[ 6.32710103e-03 6.47074357e-03 2.08291970e...,E,minor,E,minor,E,minor,E,minor
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,[[ 6.32149719e+02 -1.68478195e+02 -1.52079147e...,[[ 0.00959922 -0.00896808 0.00999433 -0.00913...,A,minor,F,major,F,major,C,major
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,[[ 2.77114404e+03 -5.87740112e+02 5.67422302e...,[[ 5.15161897e-04 1.77882321e-04 -4.16708004e...,F#,minor,F#,major,F#,major,F#,major
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,[[ 1.73931323e+03 4.87661057e+01 -1.59921829e...,[[ 8.2997873e-04 7.9957877e-05 2.2336897e-04...,A,major,A,major,A,minor,Ab,minor


Find out if any columns are not of the type float or int

In [18]:
pd.set_option('display.max_columns', None)
df_essentia_features.select_dtypes(exclude=['int64', 'float64'])

,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,lowlevel.barkbands.stdev,lowlevel.barkbands.var,lowlevel.erbbands.dmean,lowlevel.erbbands.dmean2,lowlevel.erbbands.dvar,lowlevel.erbbands.dvar2,lowlevel.erbbands.max,lowlevel.erbbands.mean,lowlevel.erbbands.median,lowlevel.erbbands.min,lowlevel.erbbands.stdev,lowlevel.erbbands.var,lowlevel.gfcc.mean,lowlevel.melbands.dmean,lowlevel.melbands.dmean2,lowlevel.melbands.dvar,lowlevel.melbands.dvar2,lowlevel.melbands.max,lowlevel.melbands.mean,lowlevel.melbands.median,lowlevel.melbands.min,lowlevel.melbands.stdev,lowlevel.melbands.var,lowlevel.melbands128.dmean,lowlevel.melbands128.dmean2,lowlevel.melbands128.dvar,lowlevel.melbands128.dvar2,lowlevel.melbands128.max,lowlevel.melbands128.mean,lowlevel.melbands128.median,lowlevel.melbands128.min,lowlevel.melbands128.stdev,lowlevel.melbands128.var,lowlevel.mfcc.mean,lowlevel.spectral_contrast_coeffs.dmean,lowlevel.spectral_contrast_coeffs.dmean2,lowlevel.spectral_contrast_coeffs.dvar,lowlevel.spectral_contrast_coeffs.dvar2,lowlevel.spectral_contrast_coeffs.max,lowlevel.spectral_contrast_coeffs.mean,lowlevel.spectral_contrast_coeffs.median,lowlevel.spectral_contrast_coeffs.min,lowlevel.spectral_contrast_coeffs.stdev,lowlevel.spectral_contrast_coeffs.var,lowlevel.spectral_contrast_valleys.dmean,lowlevel.spectral_contrast_valleys.dmean2,lowlevel.spectral_contrast_valleys.dvar,lowlevel.spectral_contrast_valleys.dvar2,lowlevel.spectral_contrast_valleys.max,lowlevel.spectral_contrast_valleys.mean,lowlevel.spectral_contrast_valleys.median,lowlevel.spectral_contrast_valleys.min,lowlevel.spectral_contrast_valleys.stdev,lowlevel.spectral_contrast_valleys.var,rhythm.beats_loudness_band_ratio.dmean,rhythm.beats_loudness_band_ratio.dmean2,rhythm.beats_loudness_band_ratio.dvar,rhythm.beats_loudness_band_ratio.dvar2,rhythm.beats_loudness_band_ratio.max,rhythm.beats_loudness_band_ratio.mean,rhythm.beats_loudness_band_ratio.median,rhythm.beats_loudness_band_ratio.min,rhythm.beats_loudness_band_ratio.stdev,rhythm.beats_loudness_band_ratio.var,tonal.hpcp.dmean,tonal.hpcp.dmean2,tonal.hpcp.dvar,tonal.hpcp.dvar2,tonal.hpcp.max,tonal.hpcp.mean,tonal.hpcp.median,tonal.hpcp.min,tonal.hpcp.stdev,tonal.hpcp.var,rhythm.beats_position,rhythm.bpm_histogram,tonal.chords_histogram,tonal.thpcp,lowlevel.gfcc.cov,lowlevel.gfcc.icov,lowlevel.mfcc.cov,lowlevel.mfcc.icov,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,[1.2053422e-04 2.1831736e-03 2.7907023e-03 2.5...,[1.7454071e-04 3.6554451e-03 4.6823896e-03 4.5...,[3.76151377e-08 2.10198741e-05 2.10502640e-05 ...,[8.0381035e-08 5.6922072e-05 5.8773549e-05 8.5...,[3.4457310e-03 6.7820773e-02 7.5503379e-02 6.9...,[2.3670899e-04 3.7311404e-03 8.1137158e-03 3.3...,[1.01748359e-04 1.92306505e-03 7.57542811e-03 ...,[4.4578975e-24 1.6160322e-23 1.6657476e-23 6.3...,[3.4085586e-04 5.8601107e-03 5.4719327e-03 5.8...,[1.1618272e-07 3.4340897e-05 2.9942046e-05 3.4...,[2.2042014e-02 6.4298314e-01 3.1456070e+00 5.4...,[3.5810590e-02 1.0805440e+00 5.3119197e+00 9.7...,[1.6460074e-03 1.7974007e+00 3.0234972e+01 9.0...,[4.0270686e-03 4.9510312e+00 8.5557724e+01 2.8...,[5.7305551e-01 2.0375187e+01 8.9572899e+01 1.2...,[3.90760601e-02 1.20288050e+00 8.54688263e+00 ...,[2.0089256e-02 7.5450528e-01 8.0763340e+00 5.6...,[7.9195445e-22 2.6390224e-21 1.1991684e-20 1.1...,[5.5168662e-02 1.6681206e+00 6.2613988e+00 1.2...,[3.0435813e-03 2.7826264e+00 3.9205112e+01 1.5...,[ -17.107492 128.29683 -121.23301 37.1...,[3.8734154e-04 1.3060000e-03 1.1690557e-03 8.9...,[6.35769742e-04 2.21744366e-03 2.08189012e-03 ...,[6.23786264e-07 5.42904763e-06 7.17484409e-06 ...,[1.61250796e-06 1.54023091e-05 2.11017159e-05 ...,[1.19236326e-02 3.92419770e-02 4.26598564e-02 ...,[6.9763616e-04 3.

Seems like there are 3 main types of column types:

1. ndarray
2. string
3. float64 or int64

Get the columns whose type is ndarray

In [19]:
pd.reset_option('display.max_columns')

string_columns = ['tonal.chords_key',
                  'tonal.chords_scale',
                  'tonal.key_edma.key',
                  'tonal.key_edma.scale',
                  'tonal.key_krumhansl.key',
                  'tonal.key_krumhansl.scale',
                  'tonal.key_temperley.key',
                  'tonal.key_temperley.scale'
                  ]

df_essentia_features_ndarray_columns = df_essentia_features.select_dtypes(exclude=['int64', 'float64'])
ndarray_columns = df_essentia_features_ndarray_columns.columns.difference(string_columns)
df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns[ndarray_columns]
df_essentia_features_ndarray_columns

,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,lowlevel.barkbands.stdev,lowlevel.barkbands.var,...,tonal.hpcp.dmean2,tonal.hpcp.dvar,tonal.hpcp.dvar2,tonal.hpcp.max,tonal.hpcp.mean,tonal.hpcp.median,tonal.hpcp.min,tonal.hpcp.stdev,tonal.hpcp.var,tonal.thpcp
0,[1.2053422e-04 2.1831736e-03 2.7907023e-03 2.5...,[1.7454071e-04 3.6554451e-03 4.6823896e-03 4.5...,[3.76151377e-08 2.10198741e-05 2.10502640e-05 ...,[8.0381035e-08 5.6922072e-05 5.8773549e-05 8.5...,[3.4457310e-03 6.7820773e-02 7.5503379e-02 6.9...,[2.3670899e-04 3.7311404e-03 8.1137158e-03 3.3...,[1.01748359e-04 1.92306505e-03 7.57542811e-03 ...,[4.4578975e-24 1.6160322e-23 1.6657476e-23 6.3...,[3.4085586e-04 5.8601107e-03 5.4719327e-03 5.8...,[1.1618272e-07 3.4340897e-05 2.9942046e-05 3.4...,...,[0.1902606 0.25763994 0.18170044 0.09408311 0...,[0.03844468 0.04496792 0.035027 0.02178529 0...,[0.09700087 0.12098674 0.10104539 0.05716944 0...,[1. 1. 1. 1. 1...,[0.3874149 0.24447472 0.10721309 0.15632775 0...,[0.06228605 0.05297357 0.02270157 0.01036106 0...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0.43815985 0.30468518 0.19752678 0.32894257 0...,[0.19198406 0.09283306 0.03901683 0.10820322 0...,[1. 0.6310411 0.27673972 0.40351507 0...
1,[4.9013726e-04 1.4123257e-02 7.3084063e-03 4.1...,[7.6726335e-04 2.1336639e-02 1.0703781e-02 6.9...,[9.27946360e-07 9.75939911e-04 3.39604769e-04 ...,[2.3075884e-06 2.1144818e-03 5.0582387e-04 2.8...,[1.6950013e-02 6.8793738e-01 4.9265939e-01 1.3...,[2.0793714e-03 7.7411823e-02 1.9229108e-02 6.0...,[1.72716519e-03 3.26213054e-02 7.26638827e-03 ...,[7.6884040e-24 9.6452232e-24 1.0116244e-23 2.1...,[1.85590365e-03 1.14251480e-01 3.73890586e-02 ...,[3.44437854e-06 1.30534014e-02 1.39794173e-03 ...,...,[0.12284187 0.12205482 0.11505249 0.05592068 0...,[0.02677063 0.02433011 0.02643875 0.01544982 0...,[0.06539019 0.06255065 0.08125681 0.04870839 0...,[1. 1. 1. 1. 1...,[0.24391279 0.23446952 0.05031401 0.01580093 0...,[0.00408666 0.00451592 0.00200037 0.00052784 0...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0.3916508 0.38068515 0.14440528 0.08976933 0...,[0.15339035 0.14492118 0.02085288 0.00805853 0...,[1. 0.656087 0.3012299 0.21130218 0...
2,[1.12729834e-03 6.19442621e-03 7.10426457e-03 ...,[1.5085544e-03 1.0527643e-02 1.0508424e-02 3.0...,[8.1122607e-06 1.3253283e-04 1.0716830e-04 7.0...,[1.4579388e-05 3.5170524e-04 2.1562108e-04 1.5...,[0.03893358 0.09716111 0.11514131 0.03183259 0...,[1.6592819e-03 8.6822817e-03 1.5993476e-02 4.2...,[6.19075727e-05 4.76877298e-03 9.66574624e-03 ...,[3.0449412e-24 1.8060825e-23 6.6988845e-24 8.9...,[4.24513174e-03 1.19597735e-02 1.79778673e-02 ...,[1.8021145e-05 1.4303617e-04 3.2320371e-04 2.1...,...,[0.3981001 0.42757905 0.23915415 0.17701466 0...,[0.08295766 0.06164335 0.05006536 0.04538609 0...,[0.16585633 0.15428595 0.13947387 0.13266544 0...,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,[0.58560866 0.38704023 0.09922621 0.05396027 0...,[0.72074664 0.36328873 0.01721864 0.00236882 0...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0.416277 0.32546043 0.19818489 0.16590182 0...,[0.17328654 0.1059245 0.03927725 0.02752341 0...,[1. 0.66091955 0.16944116 0.09214392 0...
3,[4.69437424e-07 2.34771014e-05 6.01537351e-04 ...,[8.3551004e-07 4.0238505e-05 1.0276039e-03 3.7...,[7.9561179e-13 2.7127063e-09 1.2021742e-06 1.0...,[2.1077606e-12 6.9133557e-09 3.8681392e-06 2.9...,[1.7243048e-05 6.6975842e-04 1.1746919e-02 3.7...,[4.23931766e-07 2.19649974e-05 9.46748187e-04 ...,[1.6292684e-07 6.5987815e-06 4.2298407e-04 1.8...,[4.31136624e-24 3.46033678e-23 3.58060099e-24 ...,[8.9738808e-07 5.0144037e-05 1.3454991e-03 5.0...,[8.0530537e-13 2.5144244e-09 1.8103677e-06 2.5...,...,[0.04728694 0.06397609 0.08999789 0.11920901 0...,[0.00705749 0.01249697 0.01852174 0.02452474 0...,[0.02047727 0.03337277 0.04826401 0.06129339 0...,[0.7746613 1. 1. 1.

Print one random value

In [20]:
print(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0])

[[ 4.87567624e-04  2.20520218e-04  3.65056156e-04  2.88071897e-04
   2.89849209e-04  2.31984683e-04  2.59614986e-04  2.49728269e-04
   1.29869499e-04 -2.37477856e-04  4.56099806e-04  2.42279566e-04
   1.35303795e-04]
 [ 2.20520218e-04  1.79088663e-03  1.02405704e-03  5.53560792e-04
   2.52419413e-04  9.17865778e-04 -2.81324581e-04  3.09203751e-04
   2.23003604e-04  5.24020812e-04  7.84816075e-05  5.56909072e-04
   1.58743223e-05]
 [ 3.65056156e-04  1.02405704e-03  3.39127029e-03 -5.98416264e-05
   2.03640113e-04 -1.70829575e-04  3.51056398e-04  4.75769775e-05
  -8.45025585e-04  1.38677284e-03 -1.80455274e-04  8.43082889e-05
   4.14722424e-04]
 [ 2.88071897e-04  5.53560792e-04 -5.98416264e-05  3.75327701e-03
  -6.06890651e-04  1.81736832e-03  4.74581640e-04 -8.48372991e-04
   7.22668483e-04 -6.34286029e-04  5.61190187e-04 -4.72386222e-04
  -5.11394581e-04]
 [ 2.89849209e-04  2.52419413e-04  2.03640113e-04 -6.06890651e-04
   5.87372947e-03 -3.28048714e-04  9.55037831e-04  1.24201737e-03


Flatten the columns whose values are ndarrays, like tonal.chords_histogram

Credits to https://stackoverflow.com/questions/45704999/how-to-convert-vector-wrapped-as-string-to-numpy-array-in-pandas-dataframe

In [21]:
def string_to_ndarray(str):
  return np.fromstring(str.replace('\n','')
                       .replace('[','')
                       .replace(']','')
                       .replace('  ',' '), 
                       sep=' '
                       )

In [22]:
df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns.applymap(string_to_ndarray)
df_essentia_features_ndarray_columns

/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_65026/3672827613.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns.applymap(string_to_ndarray)


,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,lowlevel.barkbands.stdev,lowlevel.barkbands.var,...,tonal.hpcp.dmean2,tonal.hpcp.dvar,tonal.hpcp.dvar2,tonal.hpcp.max,tonal.hpcp.mean,tonal.hpcp.median,tonal.hpcp.min,tonal.hpcp.stdev,tonal.hpcp.var,tonal.thpcp
0,"[0.00012053422, 0.0021831736, 0.0027907023, 0....","[0.00017454071, 0.0036554451, 0.0046823896, 0....","[3.76151377e-08, 2.10198741e-05, 2.1050264e-05...","[8.0381035e-08, 5.6922072e-05, 5.8773549e-05, ...","[0.003445731, 0.067820773, 0.075503379, 0.0697...","[0.00023670899, 0.0037311404, 0.0081137158, 0....","[0.000101748359, 0.00192306505, 0.00757542811,...","[4.4578975e-24, 1.6160322e-23, 1.6657476e-23, ...","[0.00034085586, 0.0058601107, 0.0054719327, 0....","[1.1618272e-07, 3.4340897e-05, 2.9942046e-05, ...",...,"[0.1902606, 0.25763994, 0.18170044, 0.09408311...","[0.03844468, 0.04496792, 0.035027, 0.02178529,...","[0.09700087, 0.12098674, 0.10104539, 0.0571694...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.3874149, 0.24447472, 0.10721309, 0.15632775...","[0.06228605, 0.05297357, 0.02270157, 0.0103610...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.43815985, 0.30468518, 0.19752678, 0.3289425...","[0.19198406, 0.09283306, 0.03901683, 0.1082032...","[1.0, 0.6310411, 0.27673972, 0.40351507, 0.298..."
1,"[0.00049013726, 0.014123257, 0.0073084063, 0.0...","[0.00076726335, 0.021336639, 0.010703781, 0.00...","[9.2794636e-07, 0.000975939911, 0.000339604769...","[2.3075884e-06, 0.0021144818, 0.00050582387, 0...","[0.016950013, 0.68793738, 0.49265939, 0.133314...","[0.0020793714, 0.077411823, 0.019229108, 0.006...","[0.00172716519, 0.0326213054, 0.00726638827, 0...","[7.688404e-24, 9.6452232e-24, 1.0116244e-23, 2...","[0.00185590365, 0.11425148, 0.0373890586, 0.01...","[3.44437854e-06, 0.0130534014, 0.00139794173, ...",...,"[0.12284187, 0.12205482, 0.11505249, 0.0559206...","[0.02677063, 0.02433011, 0.02643875, 0.0154498...","[0.06539019, 0.06255065, 0.08125681, 0.0487083...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.24391279, 0.23446952, 0.05031401, 0.0158009...","[0.00408666, 0.00451592, 0.00200037, 0.0005278...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.3916508, 0.38068515, 0.14440528, 0.08976933...","[0.15339035, 0.14492118, 0.02085288, 0.0080585...","[1.0, 0.656087, 0.3012299, 0.21130218, 0.05354..."
2,"[0.00112729834, 0.00619442621, 0.00710426457, ...","[0.0015085544, 0.010527643, 0.010508424, 0.003...","[8.1122607e-06, 0.00013253283, 0.0001071683, 7...","[1.4579388e-05, 0.00035170524, 0.00021562108, ...","[0.03893358, 0.09716111, 0.11514131, 0.0318325...","[0.0016592819, 0.0086822817, 0.015993476, 0.00...","[6.19075727e-05, 0.00476877298, 0.00966574624,...","[3.0449412e-24, 1.8060825e-23, 6.6988845e-24, ...","[0.00424513174, 0.0119597735, 0.0179778673, 0....","[1.8021145e-05, 0.00014303617, 0.00032320371, ...",...,"[0.3981001, 0.42757905, 0.23915415, 0.17701466...","[0.08295766, 0.06164335, 0.05006536, 0.0453860...","[0.16585633, 0.15428595, 0.13947387, 0.1326654...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.58560866, 0.38704023, 0.09922621, 0.0539602...","[0.72074664, 0.36328873, 0.01721864, 0.0023688...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.416277, 0.32546043, 0.19818489, 0.16590182,...","[0.17328654, 0.1059245, 0.03927725, 0.02752341...","[1.0, 0.66091955, 0.16944116, 0.09214392, 0.06..."
3,"[4.69437424e-07, 2.34771014e-05, 0.00060153735...","[8.3551004e-07, 4.0238505e-05, 0.0010276039, 0...","[7.9561179e-13, 2.7127063e-09, 1.2021742e-06, ...","[2.1077606e-12, 6.9133557e-09, 3.8681392e-06, ...","[1.7243048e-05, 0.00066975842, 0.011746919, 0....","[4.23931766e-07, 2.19649974e-05, 0.00094674818...","[1.6292684e-07, 6.5987815e-06, 0.00042298407, ...","[4.31136624e-24, 3.46033678e-23, 3.58060099e-2...","[8.9738808e-07, 5.0144037e-05, 0.00

Print out that same value to ensure that it is properly converted to a ndarray

In [23]:
print(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0])
print(type(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0]))

[ 4.87567624e-04  2.20520218e-04  3.65056156e-04  2.88071897e-04
  2.89849209e-04  2.31984683e-04  2.59614986e-04  2.49728269e-04
  1.29869499e-04 -2.37477856e-04  4.56099806e-04  2.42279566e-04
  1.35303795e-04  2.20520218e-04  1.79088663e-03  1.02405704e-03
  5.53560792e-04  2.52419413e-04  9.17865778e-04 -2.81324581e-04
  3.09203751e-04  2.23003604e-04  5.24020812e-04  7.84816075e-05
  5.56909072e-04  1.58743223e-05  3.65056156e-04  1.02405704e-03
  3.39127029e-03 -5.98416264e-05  2.03640113e-04 -1.70829575e-04
  3.51056398e-04  4.75769775e-05 -8.45025585e-04  1.38677284e-03
 -1.80455274e-04  8.43082889e-05  4.14722424e-04  2.88071897e-04
  5.53560792e-04 -5.98416264e-05  3.75327701e-03 -6.06890651e-04
  1.81736832e-03  4.74581640e-04 -8.48372991e-04  7.22668483e-04
 -6.34286029e-04  5.61190187e-04 -4.72386222e-04 -5.11394581e-04
  2.89849209e-04  2.52419413e-04  2.03640113e-04 -6.06890651e-04
  5.87372947e-03 -3.28048714e-04  9.55037831e-04  1.24201737e-03
 -1.41489285e-03  7.57973

In [24]:
pd.reset_option('display.max_columns')

Now that all values are proper ndarrays, let's flatten these columns, define a function to flatten a column into multiple new columns containing float64

In [25]:
def flatten_column(df, col):
  print(col)
  result_dict = {}
  num_of_new_cols = max([len(i) for i in df[col]])
  # num_of_new_cols = len(df[col][0])
  num_of_rows = len(df[col])

  for i in range(num_of_new_cols):
    result_col_name = f'{col}_{i}'
    result_dict[result_col_name] = []

  for i in range(num_of_rows):
    for j in range(num_of_new_cols):
      result_col_name = f'{col}_{j}'

      # do padding
      if j >= len(df[col][i]):
        value = 0
      else:
        value = df[col][i][j]
      
      result_dict[result_col_name].append(value)

  return pd.DataFrame(result_dict)

# test the function
df = flatten_column(df_essentia_features_ndarray_columns, 'lowlevel.mfcc.icov')
df

lowlevel.mfcc.icov


,lowlevel.mfcc.icov_0,lowlevel.mfcc.icov_1,lowlevel.mfcc.icov_2,lowlevel.mfcc.icov_3,lowlevel.mfcc.icov_4,lowlevel.mfcc.icov_5,lowlevel.mfcc.icov_6,lowlevel.mfcc.icov_7,lowlevel.mfcc.icov_8,lowlevel.mfcc.icov_9,...,lowlevel.mfcc.icov_159,lowlevel.mfcc.icov_160,lowlevel.mfcc.icov_161,lowlevel.mfcc.icov_162,lowlevel.mfcc.icov_163,lowlevel.mfcc.icov_164,lowlevel.mfcc.icov_165,lowlevel.mfcc.icov_166,lowlevel.mfcc.icov_167,lowlevel.mfcc.icov_168
0,0.000488,0.000221,0.000365,0.000288,0.000290,0.000232,0.000260,0.000250,0.000130,-0.000237,...,-0.000511,-0.001790,0.001958,-0.001566,0.000185,-0.000219,-0.000014,0.000084,-0.002813,0.011502
1,0.000613,0.000559,-0.000419,-0.000207,0.000128,0.000123,0.000322,-0.000080,-0.000197,0.000406,...,0.000744,0.001508,-0.000783,-0.002496,0.002138,-0.001049,0.000667,0.001839,-0.002898,0.016035
2,0.000253,0.000127,-0.000062,0.000111,-0.000061,-0.000424,0.000251,0.000153,-0.000440,0.000377,...,-0.000495,0.000525,0.000633,-0.000381,0.000625,-0.001389,0.001827,0.000711,-0.007506,0.021543
3,0.000480,0.000493,0.000434,0.000177,0.000185,0.000275,-0.000356,-0.000083,-0.000172,-0.000062,...,-0.000377,0.001930,-0.002871,0.000703,-0.001828,0.001128,-0.001121,0.001011,-0.000843,0.010839
4,0.000439,0.000460,0.000448,-0.000433,0.000915,-0.000662,0.000743,0.000148,-0.000120,0.000124,...,-0.000006,-0.001089,0.000579,-0.000148,0.004371,0.002411,-0.003522,0.001916,-0.018771,0.037808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,0.006327,0.006471,0.002083,0.001105,0.001394,-0.000270,-0.000898,-0.000183,-0.000209,-0.000072,...,-0.002408,0.000177,0.000564,0.000632,0.001081,0.001281,0.001699,0.003795,-0.008285,0.019939
1740,0.009599,-0.008968,0.009994,-0.009139,0.009751,-0.007795,0.007767,-0.006529,0.004203,-0.003608,...,-0.003700,0.005046,-0.010242,0.011763,-0.013494,0.011907,-0.010296,0.009289,-0.016759,0.027937
1741,0.000515,0.000178,-0.000417,-0.000190,0.000066,-0.000283,0.000242,0.000258,0.000068,-0.000514,...,0.001329,-0.003088,0.002755,0.001890,-0.002032,-0.001986,0.001056,0.002702,-0.011885,0.023021
1742,0.000830,0.000080,0.000223,0.000345,0.000436,-0.000483,-0.000570,-0.000396,0.000237,-0.000412,...,-0.003000,0.001488,-0.000533,-0.000530,0.001753,-0.000495,0.002512,0.004889,-0.010346,0.023466


Apply the function to flatten all these columns

In [26]:
ndarray_columns = df_essentia_features_ndarray_columns.columns.to_list()
df_ndarray_columns = []

for column in ndarray_columns:
  df_ndarray_column = flatten_column(df_essentia_features_ndarray_columns, column)
  df_ndarray_columns.append(df_ndarray_column)

df_essentia_features_ndarray_columns = pd.concat(df_ndarray_columns, axis=1)
df_essentia_features_ndarray_columns.insert(0, column='song_id', value=song_ids)
df_essentia_features_ndarray_columns

lowlevel.barkbands.dmean
lowlevel.barkbands.dmean2
lowlevel.barkbands.dvar
lowlevel.barkbands.dvar2
lowlevel.barkbands.max
lowlevel.barkbands.mean
lowlevel.barkbands.median
lowlevel.barkbands.min
lowlevel.barkbands.stdev
lowlevel.barkbands.var
lowlevel.erbbands.dmean
lowlevel.erbbands.dmean2
lowlevel.erbbands.dvar
lowlevel.erbbands.dvar2
lowlevel.erbbands.max
lowlevel.erbbands.mean
lowlevel.erbbands.median
lowlevel.erbbands.min
lowlevel.erbbands.stdev
lowlevel.erbbands.var
lowlevel.gfcc.cov
lowlevel.gfcc.icov
lowlevel.gfcc.mean
lowlevel.melbands.dmean
lowlevel.melbands.dmean2
lowlevel.melbands.dvar
lowlevel.melbands.dvar2
lowlevel.melbands.max
lowlevel.melbands.mean
lowlevel.melbands.median
lowlevel.melbands.min
lowlevel.melbands.stdev
lowlevel.melbands.var
lowlevel.melbands128.dmean
lowlevel.melbands128.dmean2
lowlevel.melbands128.dvar
lowlevel.melbands128.dvar2
lowlevel.melbands128.max
lowlevel.melbands128.mean
lowlevel.melbands128.median
lowlevel.melbands128.min
lowlevel.melbands128

,song_id,lowlevel.barkbands.dmean_0,lowlevel.barkbands.dmean_1,lowlevel.barkbands.dmean_2,lowlevel.barkbands.dmean_3,lowlevel.barkbands.dmean_4,lowlevel.barkbands.dmean_5,lowlevel.barkbands.dmean_6,lowlevel.barkbands.dmean_7,lowlevel.barkbands.dmean_8,...,tonal.thpcp_26,tonal.thpcp_27,tonal.thpcp_28,tonal.thpcp_29,tonal.thpcp_30,tonal.thpcp_31,tonal.thpcp_32,tonal.thpcp_33,tonal.thpcp_34,tonal.thpcp_35
0,2,1.205342e-04,0.002183,0.002791,0.002507,0.003576,0.002223,0.001575,0.000384,0.001054,...,0.112062,0.103428,0.089123,0.093243,0.123954,0.191227,0.467406,0.636457,0.522700,0.720264
1,3,4.901373e-04,0.014123,0.007308,0.004151,0.002706,0.000590,0.000150,0.000051,0.000077,...,0.058849,0.052423,0.051287,0.202352,0.300390,0.159115,0.086251,0.124976,0.172695,0.572707
2,4,1.127298e-03,0.006194,0.007104,0.002001,0.002827,0.001158,0.001887,0.000636,0.000325,...,0.045491,0.045895,0.061081,0.143019,0.241066,0.171851,0.068135,0.070274,0.146008,0.561585
3,5,4.694374e-07,0.000023,0.000602,0.002236,0.003139,0.001316,0.000660,0.000869,0.001182,...,0.165762,0.291513,0.184249,0.061519,0.038532,0.059886,0.132816,0.212119,0.231836,0.573230
4,7,3.784472e-04,0.012658,0.011005,0.004535,0.002767,0.001240,0.000889,0.000240,0.000068,...,0.080658,0.087046,0.126096,0.302461,0.753354,0.664218,0.210842,0.122887,0.142193,0.348491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,1.742373e-04,0.008582,0.002679,0.002466,0.003539,0.003790,0.001981,0.000538,0.001082,...,0.138373,0.107910,0.103334,0.203656,0.279960,0.208627,0.150923,0.200367,0.355804,0.778724
1740,1997,9.665351e-06,0.001835,0.002630,0.004954,0.005706,0.000820,0.001152,0.000947,0.001454,...,0.171188,0.182611,0.261804,0.617110,0.815704,0.424758,0.114526,0.082841,0.092267,0.503055
1741,1998,7.113585e-04,0.004704,0.004403,0.002603,0.002884,0.002336,0.001891,0.001186,0.000597,...,0.222957,0.259591,0.345576,0.569613,0.632711,0.439516,0.362685,0.357562,0.411775,0.869873
1742,1999,4.989357e-04,0.003875,0.003716,0.003070,0.003302,0.003545,0.000620,0.000627,0.001584,...,0.106994,0.143343,0.204589,0.344818,0.397066,0.306085,0.275329,0.281708,0.380624,0.843698


For the string columns, convert these categorical data into numerical data, get the dataframe with only the string columns first

In [27]:
df_essentia_features_string_columns = df_essentia_features[string_columns]
df_essentia_features_string_columns

,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,A,minor,D,minor,D,minor,A,minor
1,E,minor,A,minor,A,minor,A,minor
2,A,minor,D,minor,D,minor,A,minor
3,B,major,B,major,B,major,B,major
4,G,major,C,major,C,major,C,major
...,...,...,...,...,...,...,...,...
1739,E,minor,E,minor,E,minor,E,minor
1740,A,minor,F,major,F,major,C,major
1741,F#,minor,F#,major,F#,major,F#,major
1742,A,major,A,major,A,minor,Ab,minor


Then use cat.codes attribute to convert these categorical columns into numerical columns

In [28]:
for col in df_essentia_features_string_columns.columns:
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].astype('category')
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].cat.codes

df_essentia_features_string_columns.insert(0, column='song_id', value=song_ids)
df_essentia_features_string_columns

/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_65026/4037814126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].astype('category')
/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_65026/4037814126.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].cat.codes
/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_

,song_id,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,2,0,1,6,1,6,1,0,1
1,3,7,1,0,1,0,1,0,1
2,4,0,1,6,1,6,1,0,1
3,5,2,0,2,0,2,0,2,0
4,7,11,0,4,0,4,0,4,0
...,...,...,...,...,...,...,...,...,...
1739,1996,7,1,7,1,7,1,7,1
1740,1997,0,1,9,0,9,0,4,0
1741,1998,10,1,10,0,10,0,10,0
1742,1999,0,0,0,0,0,1,1,1


Combine the ndarray columns, string columns, and the rest of the dataframe together in one flattened dataframe with just numerical data

In [29]:
df_essentia_features_numerical_columns = df_essentia_features.select_dtypes(include=['int64', 'float64'])

df_temp = pd.merge(df_essentia_features_numerical_columns, df_essentia_features_ndarray_columns, how='inner', on='song_id')
df_essentia_features_flattened = pd.merge(df_temp, df_essentia_features_string_columns, how='inner', on='song_id')

df_essentia_features_flattened

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,tonal.thpcp_34,tonal.thpcp_35,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,0.522700,0.720264,0,1,6,1,6,1,0,1
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,0.172695,0.572707,7,1,0,1,0,1,0,1
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,0.146008,0.561585,0,1,6,1,6,1,0,1
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,0.231836,0.573230,2,0,2,0,2,0,2,0
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,0.142193,0.348491,11,0,4,0,4,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,0.355804,0.778724,7,1,7,1,7,1,7,1
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,0.092267,0.503055,0,1,9,0,9,0,4,0
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,0.411775,0.869873,10,1,10,0,10,0,10,0
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,0.380624,0.843698,0,0,0,0,0,1,1,1


Export the flattened Essentia features dataset

In [30]:
df_essentia_features_flattened.to_csv(get_deam_path('processed/features/essentia_features_flattened.csv'))

## Create separate, more distinct features dataframes extracted by Essentia

Import the features lsit

In [31]:
from deam_essentia_best_features import *

### Get song ids for songs in 2013 and 2014

In [32]:
audio_path = get_deam_path('MEMD_audio')
song_ids_temp = []

# Iterate through all files in the directory
for filename in os.listdir(audio_path):
    # Check if the path is a file (not a subdirectory)
    if os.path.isfile(os.path.join(audio_path, filename)):
        song_ids_temp.append(int(filename[:-4]))

song_ids_temp.sort()
song_ids = []

# remove all song_ids from 2015 (song_id 2001 onwards)
for song_id in song_ids_temp:
    if song_id <= 2000:
        song_ids.append(song_id)

### Create separate feature dataset for best features for building arousal regressor
According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [33]:
df_essentia_best_arousal_features = df_essentia_features_flattened[deam_essentia_best_arousal_features_flattened]
df_essentia_best_arousal_features.insert(0, 'song_id', song_ids)
df_essentia_best_arousal_features.to_csv(get_deam_path('processed/features/essentia_best_arousal_features.csv'))

Import the best feature dataset .csv for arousal

In [34]:
df_essentia_best_arousal_features = pd.read_csv(get_deam_path('processed/features/essentia_best_arousal_features.csv'))

# drop Unnamed:0 column
df_essentia_best_arousal_features = df_essentia_best_arousal_features[df_essentia_best_arousal_features.columns[1:]]

df_essentia_best_arousal_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.dmean,lowlevel.barkbands_spread.dmean2,lowlevel.barkbands_spread.dvar,lowlevel.barkbands_spread.dvar2,lowlevel.barkbands_spread.max,lowlevel.barkbands_spread.mean,lowlevel.barkbands_spread.median,lowlevel.barkbands_spread.min,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.960248,4.550666,7.235502,37.865948,95.265396,94.332970,15.924681,14.074156,1.380657,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,0.577547,1.150496,1.781423,8.920459,20.415138,65.661095,4.013522,2.604582,0.105044,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,0.978169,4.410288,6.733066,22.211205,48.142967,82.455566,15.952667,14.231352,0.854718,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,0.902877,2.369953,3.454134,19.432764,34.974041,86.441460,9.326209,7.617138,0.611234,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,0.978392,2.122688,3.430682,29.966152,82.799446,91.631195,3.206823,1.772354,0.222541,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.652898,5.749208,10.690978,24.899950,40.925282,13.047432,11.716851,1.945637,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,0.985275,3.603367,5.866543,12.558491,31.335993,49.654797,15.037008,14.095716,1.787386,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,0.899922,6.775062,10.153251,75.743263,166.500168,106.074104,22.843222,17.990805,2.192569,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,0.962358,3.589267,5.683703,14.578965,35.868526,69.180779,13.325944,12.539411,2.097204,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


### Create separate feature dataset for best features for building valence regressor
According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [35]:
df_essentia_best_valence_features = df_essentia_features_flattened[deam_essentia_best_valence_features_flattened]
df_essentia_best_valence_features.insert(0, 'song_id', song_ids)
df_essentia_best_valence_features.to_csv(get_deam_path('processed/features/essentia_best_valence_features.csv'))

Import the best feature dataset .csv for valence

In [36]:
df_essentia_best_valence_features = pd.read_csv(get_deam_path('processed/features/essentia_best_valence_features.csv'))

# drop Unnamed:0 column
df_essentia_best_valence_features = df_essentia_best_valence_features[df_essentia_best_valence_features.columns[1:]]

df_essentia_best_valence_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


### Create separate feature dataset for best overall features for detecting both arousal and valence

According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [37]:
df_essentia_best_overall_features = df_essentia_features_flattened[deam_essentia_best_overall_features_flattened]
df_essentia_best_overall_features.insert(0, 'song_id', song_ids)
df_essentia_best_overall_features.to_csv(get_deam_path('processed/features/essentia_best_overall_features.csv'))

Import the best feature dataset .csv overall 

In [38]:
df_essentia_best_overall_features = pd.read_csv(get_deam_path('processed/features/essentia_best_overall_features.csv'))

# drop Unnamed:0 column
df_essentia_best_overall_features = df_essentia_best_overall_features[df_essentia_best_overall_features.columns[1:]]

df_essentia_best_overall_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


## Integrate Essentia features into openSMILE features

Import openSMILE featuresets .csv

In [39]:
df_deam_opensmile_compare2016_features = pd.read_csv(get_deam_path('processed/features/opensmile_compare2016_features.csv'))
df_deam_opensmile_emobase_features = pd.read_csv(get_deam_path('processed/features/opensmile_emobase_features.csv'))
df_deam_opensmile_gemaps_features = pd.read_csv(get_deam_path('processed/features/opensmile_gemaps_features.csv'))
df_deam_opensmile_egemaps_features = pd.read_csv(get_deam_path('processed/features/opensmile_egemaps_features.csv'))

# drop Unnamed:0 column
df_deam_opensmile_compare2016_features = df_deam_opensmile_compare2016_features[df_deam_opensmile_compare2016_features.columns[1:]]
df_deam_opensmile_emobase_features = df_deam_opensmile_emobase_features[df_deam_opensmile_emobase_features.columns[1:]]
df_deam_opensmile_gemaps_features = df_deam_opensmile_gemaps_features[df_deam_opensmile_gemaps_features.columns[1:]]
df_deam_opensmile_egemaps_features = df_deam_opensmile_egemaps_features[df_deam_opensmile_egemaps_features.columns[1:]]

In [40]:
df_deam_opensmile_compare2016_features

,song_id,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,4.487072,0.499444,0.000000,1.693069,2.279854,2.647825,0.586785,0.367970,0.954755,...,7.324224,0.534654,2.096779,2.095753,20.000000,0.647608,100.58043,50.135098,99.799490,49.985157
1,3,2.939642,0.417260,0.000000,0.935776,1.065326,1.215337,0.129550,0.150011,0.279561,...,6.627462,0.489874,1.832508,1.833608,-19.999998,0.570708,80.85080,43.208675,80.717380,45.814630
2,4,4.937490,0.786922,0.000000,2.454838,2.963577,3.478416,0.508740,0.514839,1.023579,...,8.150249,0.506702,2.152296,2.151656,20.000000,0.624459,96.52063,51.825253,92.622505,45.681725
3,5,5.179635,0.450400,0.000000,1.997853,2.568133,3.278233,0.570280,0.710101,1.280380,...,7.644168,0.486110,2.054498,2.052557,19.999930,0.592607,89.58958,48.105022,85.724660,47.280334
4,7,3.889954,0.151812,0.000000,1.453349,1.739884,2.153651,0.286535,0.413767,0.700302,...,7.221351,0.562967,1.721128,1.722401,-19.999989,0.591437,80.62400,39.327477,79.446900,39.143200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,1.748674,0.459159,0.705987,2.456600,2.611855,2.864886,0.155254,0.253031,0.408286,...,7.014015,0.592036,1.998147,1.996065,19.999868,0.566083,93.15075,44.501705,96.057270,44.310146
1740,1997,1.770800,0.082795,0.178277,1.684116,1.875371,2.086306,0.191255,0.210935,0.402190,...,8.635638,0.530015,2.202010,2.199063,19.999430,0.565890,101.92789,55.691982,102.434510,54.795270
1741,1998,3.346390,0.490318,0.120855,2.030153,2.356498,2.708012,0.326345,0.351515,0.677860,...,7.182379,0.635415,1.889074,1.887913,19.999996,0.551982,86.53014,42.108500,89.627280,44.030266
1742,1999,3.149409,0.754730,0.432228,1.714522,1.995686,2.405440,0.281164,0.409754,0.690918,...,7.847402,0.656611,1.920955,1.920288,20.000000,0.513915,95.08800,46.408382,94.678055,46.461388


In [41]:
df_deam_opensmile_emobase_features

,song_id,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,0.000100,0.000000e+00,0.000100,3821.0,0.0,0.000003,9.746145e-11,0.000003,0.000002,...,34.224740,5.851549,0.815608,11.220467,-2.025291,0.0,1.749724,2.025291,1.749724,3.775015
1,3,0.000051,0.000000e+00,0.000051,2809.0,0.0,0.000005,1.012040e-10,0.000004,0.000004,...,11.536975,3.397905,1.259240,29.973100,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,4,0.000088,0.000000e+00,0.000088,1800.0,0.0,0.000011,5.901333e-11,0.000011,0.000008,...,31.980932,5.655746,0.692436,9.787511,-1.309070,0.0,1.036872,1.309070,1.036872,2.345942
3,5,0.000060,0.000000e+00,0.000060,488.0,0.0,0.000006,-3.841752e-10,0.000007,0.000005,...,33.398550,5.780117,0.625557,9.257450,-1.881530,0.0,1.791142,1.881530,1.791142,3.672672
4,7,0.000113,0.000000e+00,0.000113,1061.0,0.0,0.000020,-4.199130e-11,0.000020,0.000011,...,23.441298,4.842434,0.802300,13.485694,-1.159967,0.0,1.114847,1.159967,1.114847,2.274814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.000023,1.535268e-08,0.000023,2355.0,548.0,0.000005,-8.072022e-11,0.000005,0.000003,...,8.894216,2.983331,7.053555,145.043520,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1740,1997,0.000012,1.499340e-08,0.000012,3905.0,4487.0,0.000002,-4.315512e-11,0.000002,0.000001,...,2.125242,1.458878,1.890596,27.465874,-0.168976,0.0,0.000000,0.168976,0.000000,0.168976
1741,1998,0.000026,1.076092e-08,0.000026,3411.0,3137.0,0.000003,-1.908036e-10,0.000004,0.000003,...,11.348680,3.371960,0.621677,18.409992,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1742,1999,0.000030,5.323769e-09,0.000030,4167.0,1089.0,0.000002,9.297152e-11,0.000002,0.000002,...,7.925314,2.818051,3.872501,60.610363,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [42]:
df_deam_opensmile_gemaps_features

,song_id,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,22.932022,0.189338,19.655570,23.607199,24.485361,4.829792,124.252335,106.078514,60.294655,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,3,16.164516,0.178617,14.101058,15.239481,18.985416,4.884358,85.965515,134.082820,62.976208,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,4,22.580710,0.329062,16.054005,23.823282,24.116590,8.062586,299.053960,463.630200,163.715870,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,5,29.905169,0.264543,22.387814,30.137478,37.959970,15.572155,179.485930,169.986390,111.476910,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,7,18.706436,0.172212,15.197102,18.928482,21.956436,6.759335,88.499120,122.277680,51.647667,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,19.343260,0.212107,17.190958,19.071331,19.977148,2.786190,114.446846,127.966194,86.548430,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,1997,22.913340,0.271282,18.283949,21.660767,26.187181,7.903233,220.857970,371.026340,115.580100,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,1998,20.718222,0.323465,15.797006,19.021437,27.441607,11.644601,207.201580,221.265670,203.805970,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,1999,19.688591,0.249436,16.933563,19.086851,21.965055,5.031492,175.598650,295.851320,154.269360,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [43]:
df_deam_opensmile_egemaps_features

,song_id,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,22.932022,0.189338,19.655570,23.607199,24.485361,4.829792,124.252335,106.078514,60.294655,...,-0.024486,-0.007114,2.023631,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573,-19.159882
1,3,16.164516,0.178617,14.101058,15.239481,18.985416,4.884358,85.965515,134.082820,62.976208,...,-0.104570,-0.008604,2.370560,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012,-17.587570
2,4,22.580710,0.329062,16.054005,23.823282,24.116590,8.062586,299.053960,463.630200,163.715870,...,-0.052437,-0.006673,3.990483,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689,-14.016479
3,5,29.905169,0.264543,22.387814,30.137478,37.959970,15.572155,179.485930,169.986390,111.476910,...,0.054624,-0.023808,2.539289,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987,-16.089987
4,7,18.706436,0.172212,15.197102,18.928482,21.956436,6.759335,88.499120,122.277680,51.647667,...,-0.055579,-0.009088,4.034995,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756,-11.279402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,19.343260,0.212107,17.190958,19.071331,19.977148,2.786190,114.446846,127.966194,86.548430,...,-0.004562,-0.020547,2.735432,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377,-17.564531
1740,1997,22.913340,0.271282,18.283949,21.660767,26.187181,7.903233,220.857970,371.026340,115.580100,...,-0.003737,-0.019565,1.726589,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021,-21.888250
1741,1998,20.718222,0.323465,15.797006,19.021437,27.441607,11.644601,207.201580,221.265670,203.805970,...,-0.035106,-0.014033,2.011096,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816,-18.990843
1742,1999,19.688591,0.249436,16.933563,19.086851,21.965055,5.031492,175.598650,295.851320,154.269360,...,-0.012904,-0.014340,2.047864,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784,-20.438614


Integrate Essentia all features into openSMILE ComParE2016 features

In [44]:
df_deam_integrated_essentia_all_opensmile_compare2016 = pd.merge(df_essentia_features_flattened, df_deam_opensmile_compare2016_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_compare2016.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_compare2016.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_compare2016

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,7.324224,0.534654,2.096779,2.095753,20.000000,0.647608,100.58043,50.135098,99.799490,49.985157
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,6.627462,0.489874,1.832508,1.833608,-19.999998,0.570708,80.85080,43.208675,80.717380,45.814630
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,8.150249,0.506702,2.152296,2.151656,20.000000,0.624459,96.52063,51.825253,92.622505,45.681725
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,7.644168,0.486110,2.054498,2.052557,19.999930,0.592607,89.58958,48.105022,85.724660,47.280334
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,7.221351,0.562967,1.721128,1.722401,-19.999989,0.591437,80.62400,39.327477,79.446900,39.143200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,7.014015,0.592036,1.998147,1.996065,19.999868,0.566083,93.15075,44.501705,96.057270,44.310146
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,8.635638,0.530015,2.202010,2.199063,19.999430,0.565890,101.92789,55.691982,102.434510,54.795270
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,7.182379,0.635415,1.889074,1.887913,19.999996,0.551982,86.53014,42.108500,89.627280,44.030266
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,7.847402,0.656611,1.920955,1.920288,20.000000,0.513915,95.08800,46.408382,94.678055,46.461388


In [45]:
df_deam_integrated_essentia_all_opensmile_compare2016.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_compare2016_features.csv'))

Integrate Essentia all features into openSMILE emobase features

In [46]:
df_deam_integrated_essentia_all_opensmile_emobase = pd.merge(df_essentia_features_flattened, df_deam_opensmile_emobase_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_emobase.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_emobase.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_emobase

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,34.224740,5.851549,0.815608,11.220467,-2.025291,0.0,1.749724,2.025291,1.749724,3.775015
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,11.536975,3.397905,1.259240,29.973100,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,31.980932,5.655746,0.692436,9.787511,-1.309070,0.0,1.036872,1.309070,1.036872,2.345942
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,33.398550,5.780117,0.625557,9.257450,-1.881530,0.0,1.791142,1.881530,1.791142,3.672672
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,23.441298,4.842434,0.802300,13.485694,-1.159967,0.0,1.114847,1.159967,1.114847,2.274814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,8.894216,2.983331,7.053555,145.043520,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,2.125242,1.458878,1.890596,27.465874,-0.168976,0.0,0.000000,0.168976,0.000000,0.168976
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,11.348680,3.371960,0.621677,18.409992,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,7.925314,2.818051,3.872501,60.610363,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [47]:
df_deam_integrated_essentia_all_opensmile_emobase.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_emobase_features.csv'))

Integrate Essentia all features into openSMILE GeMAPS features

In [48]:
df_deam_integrated_essentia_all_opensmile_gemaps = pd.merge(df_essentia_features_flattened, df_deam_opensmile_gemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_gemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_gemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_gemaps

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [49]:
df_deam_integrated_essentia_all_opensmile_gemaps.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_gemaps_features.csv'))

Integrate Essentia all features into openSMILE eGeMAPS features

In [50]:
df_deam_integrated_essentia_all_opensmile_egemaps = pd.merge(df_essentia_features_flattened, df_deam_opensmile_egemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_egemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_egemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_egemaps

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,0.960248,2.258284,3.723765,4.063393,11.560330,22.870403,11.212613,10.802777,2.674608,...,-0.024486,-0.007114,2.023631,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573,-19.159882
1,3,0.577547,1.755316,2.888824,3.826239,10.070616,26.567934,16.432838,16.015923,4.081916,...,-0.104570,-0.008604,2.370560,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012,-17.587570
2,4,0.978169,2.483856,4.020749,5.345178,14.364439,22.231218,10.575248,10.239008,2.738014,...,-0.052437,-0.006673,3.990483,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689,-14.016479
3,5,0.902877,2.403809,3.848961,4.573623,11.834862,25.005966,11.478149,10.657066,3.504722,...,0.054624,-0.023808,2.539289,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987,-16.089987
4,7,0.978392,1.974828,3.087666,5.040214,11.790000,24.543375,15.758171,15.867741,2.866729,...,-0.055579,-0.009088,4.034995,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756,-11.279402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.163405,5.194232,8.003455,18.861530,23.048399,10.288649,9.318178,3.273469,...,-0.004562,-0.020547,2.735432,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377,-17.564531
1740,1997,0.985275,2.380977,3.982027,4.592145,12.498706,21.505104,10.112698,9.651812,3.108957,...,-0.003737,-0.019565,1.726589,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021,-21.888250
1741,1998,0.899922,2.692476,4.314886,6.035235,14.309047,24.130383,9.502740,8.967997,2.275360,...,-0.035106,-0.014033,2.011096,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816,-18.990843
1742,1999,0.962358,2.572920,4.367725,4.447225,12.239031,21.957045,9.768277,9.358503,3.522282,...,-0.012904,-0.014340,2.047864,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784,-20.438614


In [51]:
df_deam_integrated_essentia_all_opensmile_egemaps.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_egemaps_features.csv'))

Integrate Essentia best overall features into openSMILE ComParE2016 features

In [52]:
df_deam_integrated_essentia_best_overall_opensmile_compare2016 = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_compare2016_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_compare2016.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_compare2016.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_compare2016

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,7.324224,0.534654,2.096779,2.095753,20.000000,0.647608,100.58043,50.135098,99.799490,49.985157
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,6.627462,0.489874,1.832508,1.833608,-19.999998,0.570708,80.85080,43.208675,80.717380,45.814630
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,8.150249,0.506702,2.152296,2.151656,20.000000,0.624459,96.52063,51.825253,92.622505,45.681725
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,7.644168,0.486110,2.054498,2.052557,19.999930,0.592607,89.58958,48.105022,85.724660,47.280334
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,7.221351,0.562967,1.721128,1.722401,-19.999989,0.591437,80.62400,39.327477,79.446900,39.143200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,7.014015,0.592036,1.998147,1.996065,19.999868,0.566083,93.15075,44.501705,96.057270,44.310146
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,8.635638,0.530015,2.202010,2.199063,19.999430,0.565890,101.92789,55.691982,102.434510,54.795270
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,7.182379,0.635415,1.889074,1.887913,19.999996,0.551982,86.53014,42.108500,89.627280,44.030266
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,7.847402,0.656611,1.920955,1.920288,20.000000,0.513915,95.08800,46.408382,94.678055,46.461388


In [53]:
df_deam_integrated_essentia_best_overall_opensmile_compare2016.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_compare2016_features.csv'))

Integrate Essentia best overall features into openSMILE emobase features

In [54]:
df_deam_integrated_essentia_best_overall_opensmile_emobase = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_emobase_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_emobase.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_emobase.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_emobase

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,34.224740,5.851549,0.815608,11.220467,-2.025291,0.0,1.749724,2.025291,1.749724,3.775015
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,11.536975,3.397905,1.259240,29.973100,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,31.980932,5.655746,0.692436,9.787511,-1.309070,0.0,1.036872,1.309070,1.036872,2.345942
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,33.398550,5.780117,0.625557,9.257450,-1.881530,0.0,1.791142,1.881530,1.791142,3.672672
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,23.441298,4.842434,0.802300,13.485694,-1.159967,0.0,1.114847,1.159967,1.114847,2.274814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,8.894216,2.983331,7.053555,145.043520,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,2.125242,1.458878,1.890596,27.465874,-0.168976,0.0,0.000000,0.168976,0.000000,0.168976
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,11.348680,3.371960,0.621677,18.409992,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,7.925314,2.818051,3.872501,60.610363,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [55]:
df_deam_integrated_essentia_best_overall_opensmile_emobase.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_emobase_features.csv'))

Integrate Essentia best overall features into openSMILE GeMAPS features

In [56]:
df_deam_integrated_essentia_best_overall_opensmile_gemaps = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_gemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_gemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_gemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_gemaps

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [57]:
df_deam_integrated_essentia_best_overall_opensmile_gemaps.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_gemaps_features.csv'))

Integrate Essentia best overall features into openSMILE eGeMAPS features

In [58]:
df_deam_integrated_essentia_best_overall_opensmile_egemaps = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_egemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_egemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_egemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_egemaps

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,-0.024486,-0.007114,2.023631,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573,-19.159882
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,-0.104570,-0.008604,2.370560,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012,-17.587570
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,-0.052437,-0.006673,3.990483,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689,-14.016479
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,0.054624,-0.023808,2.539289,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987,-16.089987
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,-0.055579,-0.009088,4.034995,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756,-11.279402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,-0.004562,-0.020547,2.735432,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377,-17.564531
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,-0.003737,-0.019565,1.726589,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021,-21.888250
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,-0.035106,-0.014033,2.011096,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816,-18.990843
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,-0.012904,-0.014340,2.047864,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784,-20.438614


In [59]:
df_deam_integrated_essentia_best_overall_opensmile_egemaps.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_egemaps_features.csv'))

## Data Scaling

Import relevant libraries

In [60]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Data Standardisation

In [61]:
scaler = StandardScaler()

Essentia All + openSMILE ComParE2016

In [62]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_compare2016_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_compare2016), columns=df_deam_integrated_essentia_all_opensmile_compare2016.columns)

df_deam_integrated_essentia_all_opensmile_compare2016_standardised = df_deam_integrated_essentia_all_opensmile_compare2016_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_compare2016_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_compare2016_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_all_opensmile_compare2016_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,0.568151,-0.495162,-0.392697,-0.744706,-0.546485,-0.870470,-0.239716,-0.144909,-0.751646,...,-0.631867,-0.365685,-0.279802,-0.282638,1.033912,1.253936,0.258343,-0.101681,0.267050,-0.103648
1,3,-1.189985,-1.329502,-1.219457,-0.832461,-0.787198,1.027719,1.839969,1.739902,0.983723,...,-0.952109,-1.043699,-0.949817,-0.947268,-0.967357,0.042120,-1.127785,-0.883942,-1.084239,-0.561874
2,4,0.650480,-0.120976,-0.098623,-0.270405,-0.093387,-1.198606,-0.493636,-0.348739,-0.673459,...,-0.252214,-0.788912,-0.139047,-0.140901,1.033912,0.889151,-0.026883,0.089203,-0.241184,-0.576476
3,5,0.304588,-0.253761,-0.268728,-0.555905,-0.502125,0.225857,-0.133929,-0.197590,0.271977,...,-0.484816,-1.100697,-0.386997,-0.392154,1.033909,0.387215,-0.513832,-0.330955,-0.729651,-0.400833
4,7,0.651505,-0.965369,-1.022563,-0.383251,-0.509374,-0.011622,1.571188,1.686327,-0.514739,...,-0.679149,0.063000,-1.232202,-1.229221,-0.967357,0.368769,-1.143719,-1.322279,-1.174207,-1.294879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.675713,1.006280,1.063360,0.713243,0.633269,-0.779093,-0.607815,-0.681665,-0.013183,...,-0.774443,0.503124,-0.529866,-0.535381,1.033905,-0.030763,-0.263638,-0.737909,0.002047,-0.727175
1740,1997,0.683125,-0.291635,-0.136966,-0.549051,-0.394859,-1.571368,-0.677912,-0.561040,-0.216045,...,-0.029122,-0.435924,-0.013004,-0.020709,1.033884,-0.033798,0.353010,0.525906,0.453648,0.424850
1741,1998,0.291013,0.225089,0.192632,-0.015061,-0.102338,-0.223639,-0.920913,-0.808273,-1.243963,...,-0.697061,1.159923,-0.806402,-0.809586,1.033912,-0.252968,-0.728777,-1.008194,-0.453289,-0.757926
1742,1999,0.577845,0.026765,0.244953,-0.602676,-0.436818,-1.339357,-0.815125,-0.667085,0.293630,...,-0.391407,1.480851,-0.725574,-0.727505,1.033912,-0.852857,-0.127534,-0.522571,-0.095621,-0.490813


Essentia All + openSMILE emobase

In [63]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_emobase_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_emobase), columns=df_deam_integrated_essentia_all_opensmile_emobase.columns)

df_deam_integrated_essentia_all_opensmile_emobase_standardised = df_deam_integrated_essentia_all_opensmile_emobase_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_emobase_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_emobase_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_all_opensmile_emobase_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,0.568151,-0.495162,-0.392697,-0.744706,-0.546485,-0.870470,-0.239716,-0.144909,-0.751646,...,1.121299,1.099986,-0.339925,-0.368315,-1.495692,0.139276,1.251933,1.502738,1.245620,1.382243
1,3,-1.189985,-1.329502,-1.219457,-0.832461,-0.787198,1.027719,1.839969,1.739902,0.983723,...,-0.751053,-0.671540,-0.187318,-0.107478,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970
2,4,0.650480,-0.120976,-0.098623,-0.270405,-0.093387,-1.198606,-0.493636,-0.348739,-0.673459,...,0.936125,0.958616,-0.382295,-0.388247,-0.643263,0.139276,0.393597,0.647609,0.390021,0.522072
3,5,0.304588,-0.253761,-0.268728,-0.555905,-0.502125,0.225857,-0.133929,-0.197590,0.271977,...,1.053116,1.048412,-0.405301,-0.395619,-1.324591,0.139276,1.301805,1.331095,1.295332,1.320642
4,7,0.651505,-0.965369,-1.022563,-0.383251,-0.509374,-0.011622,1.571188,1.686327,-0.514739,...,0.231374,0.371407,-0.344503,-0.336807,-0.465805,0.139276,0.487485,0.469588,0.483610,0.479259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.675713,1.006280,1.063360,0.713243,0.633269,-0.779093,-0.607815,-0.681665,-0.013183,...,-0.969152,-0.970862,1.805891,1.493076,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970
1740,1997,0.683125,-0.291635,-0.136966,-0.549051,-0.394859,-1.571368,-0.677912,-0.561040,-0.216045,...,-1.527775,-2.071514,0.029865,-0.142352,0.713648,0.139276,-0.854889,-0.713601,-0.854482,-0.788262
1741,1998,0.291013,0.225089,0.192632,-0.015061,-0.102338,-0.223639,-0.920913,-0.808273,-1.243963,...,-0.766593,-0.690273,-0.406636,-0.268314,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970
1742,1999,0.577845,0.026765,0.244953,-0.602676,-0.436818,-1.339357,-0.815125,-0.667085,0.293630,...,-1.049113,-1.090194,0.711628,0.318666,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970


Essentia All + openSMILE GeMAPS

In [64]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_gemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_gemaps), columns=df_deam_integrated_essentia_all_opensmile_gemaps.columns)

df_deam_integrated_essentia_all_opensmile_gemaps_standardised = df_deam_integrated_essentia_all_opensmile_gemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_gemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_gemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_gemaps_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,0.568151,-0.495162,-0.392697,-0.744706,-0.546485,-0.870470,-0.239716,-0.144909,-0.751646,...,0.287564,-0.262130,-0.181091,0.668531,-0.555169,-0.967327,-0.075432,0.439999,-0.440097,-0.377481
1,3,-1.189985,-1.329502,-1.219457,-0.832461,-0.787198,1.027719,1.839969,1.739902,0.983723,...,-1.745983,1.481523,-1.835793,0.496337,-0.932362,0.263018,-0.389877,-0.474327,0.570241,0.450156
2,4,0.650480,-0.120976,-0.098623,-0.270405,-0.093387,-1.198606,-0.493636,-0.348739,-0.673459,...,0.025746,-0.164946,-0.758607,0.719491,0.005327,0.196521,-0.381856,-0.512572,-0.174826,-0.303339
3,5,0.304588,-0.253761,-0.268728,-0.555905,-0.502125,0.225857,-0.133929,-0.197590,0.271977,...,0.296808,-0.540060,1.453477,-1.260938,-0.603349,-1.000287,-0.015498,0.340968,-0.418676,-0.300240
4,7,0.651505,-0.965369,-1.022563,-0.383251,-0.509374,-0.011622,1.571188,1.686327,-0.514739,...,-2.342634,1.977728,-0.823529,0.440421,1.121683,-0.917488,-0.139595,0.293586,-0.372262,-0.240131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.675713,1.006280,1.063360,0.713243,0.633269,-0.779093,-0.607815,-0.681665,-0.013183,...,0.343948,-0.753080,0.230580,-0.884061,1.768270,-0.434687,-0.329498,-0.316919,-0.508329,-0.462869
1740,1997,0.683125,-0.291635,-0.136966,-0.549051,-0.394859,-1.571368,-0.677912,-0.561040,-0.216045,...,0.222617,-0.444289,0.247618,-0.770541,1.076880,-0.550845,-0.310073,-0.300887,-0.344771,-0.226950
1741,1998,0.291013,0.225089,0.192632,-0.015061,-0.102338,-0.223639,-0.920913,-0.808273,-1.243963,...,0.374784,-0.346391,-0.400532,-0.131125,1.027496,0.996925,-0.404940,-0.493444,-0.080049,-0.197836
1742,1999,0.577845,0.026765,0.244953,-0.602676,-0.436818,-1.339357,-0.815125,-0.667085,0.293630,...,0.146768,-0.269664,0.058214,-0.166695,0.138567,0.629971,-0.396049,-0.541690,-0.219187,-0.239845


Essentia All + openSMILE eGeMAPS

In [65]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_egemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_egemaps), columns=df_deam_integrated_essentia_all_opensmile_egemaps.columns)

df_deam_integrated_essentia_all_opensmile_egemaps_standardised = df_deam_integrated_essentia_all_opensmile_egemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_egemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_egemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_egemaps_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,0.568151,-0.495162,-0.392697,-0.744706,-0.546485,-0.870470,-0.239716,-0.144909,-0.751646,...,-0.181091,0.668531,-0.062375,-0.555169,-0.967327,-0.075432,0.439999,-0.440097,-0.377481,-0.057938
1,3,-1.189985,-1.329502,-1.219457,-0.832461,-0.787198,1.027719,1.839969,1.739902,0.983723,...,-1.835793,0.496337,0.172590,-0.932362,0.263018,-0.389877,-0.474327,0.570241,0.450156,0.340489
2,4,0.650480,-0.120976,-0.098623,-0.270405,-0.093387,-1.198606,-0.493636,-0.348739,-0.673459,...,-0.758607,0.719491,1.269721,0.005327,0.196521,-0.381856,-0.512572,-0.174826,-0.303339,1.245411
3,5,0.304588,-0.253761,-0.268728,-0.555905,-0.502125,0.225857,-0.133929,-0.197590,0.271977,...,1.453477,-1.260938,0.286866,-0.603349,-1.000287,-0.015498,0.340968,-0.418676,-0.300240,0.719980
4,7,0.651505,-0.965369,-1.022563,-0.383251,-0.509374,-0.011622,1.571188,1.686327,-0.514739,...,-0.823529,0.440421,1.299868,1.121683,-0.917488,-0.139595,0.293586,-0.372262,-0.240131,1.938993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.675713,1.006280,1.063360,0.713243,0.633269,-0.779093,-0.607815,-0.681665,-0.013183,...,0.230580,-0.884061,0.419708,1.768270,-0.434687,-0.329498,-0.316919,-0.508329,-0.462869,0.346327
1740,1997,0.683125,-0.291635,-0.136966,-0.549051,-0.394859,-1.571368,-0.677912,-0.561040,-0.216045,...,0.247618,-0.770541,-0.263554,1.076880,-0.550845,-0.310073,-0.300887,-0.344771,-0.226950,-0.749313
1741,1998,0.291013,0.225089,0.192632,-0.015061,-0.102338,-0.223639,-0.920913,-0.808273,-1.243963,...,-0.400532,-0.131125,-0.070865,1.027496,0.996925,-0.404940,-0.493444,-0.080049,-0.197836,-0.015104
1742,1999,0.577845,0.026765,0.244953,-0.602676,-0.436818,-1.339357,-0.815125,-0.667085,0.293630,...,0.058214,-0.166695,-0.045963,0.138567,0.629971,-0.396049,-0.541690,-0.219187,-0.239845,-0.381972


Essentia Best Overall + openSMILE ComParE2016

In [66]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_compare2016), columns=df_deam_integrated_essentia_best_overall_opensmile_compare2016.columns)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised = df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,-0.092381,-0.078572,-0.025137,-0.024931,-0.051278,-0.058571,-0.055109,0.361562,-0.018698,...,-0.631867,-0.365685,-0.279802,-0.282638,1.033912,1.253936,0.258343,-0.101681,0.267050,-0.103648
1,3,0.073364,0.050761,-0.025077,-0.024890,-0.020349,1.479718,3.014248,0.583826,0.056605,...,-0.952109,-1.043699,-0.949817,-0.947268,-0.967357,0.042120,-1.127785,-0.883942,-1.084239,-0.561874
2,4,-0.119333,-0.107505,-0.025174,-0.024965,-0.083147,-0.208423,-0.075820,0.053745,-0.094185,...,-0.252214,-0.788912,-0.139047,-0.140901,1.033912,0.889151,-0.026883,0.089203,-0.241184,-0.576476
3,5,-0.163481,-0.139436,-0.025195,-0.024978,-0.094325,-0.262338,-0.148097,-0.493194,-0.110313,...,-0.484816,-1.100697,-0.386997,-0.392154,1.033909,0.387215,-0.513832,-0.330955,-0.729651,-0.400833
4,7,0.721637,0.521923,-0.024526,-0.024479,0.015265,2.154544,4.094221,0.492728,0.184390,...,-0.679149,0.063000,-1.232202,-1.229221,-0.967357,0.368769,-1.143719,-1.322279,-1.174207,-1.294879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,-0.154993,-0.129360,-0.025193,-0.024975,-0.100449,-0.321333,-0.138527,0.145668,-0.130138,...,-0.774443,0.503124,-0.529866,-0.535381,1.033905,-0.030763,-0.263638,-0.737909,0.002047,-0.727175
1740,1997,-0.169892,-0.137550,-0.025197,-0.024978,-0.102115,-0.346532,-0.290196,-0.205395,-0.126981,...,-0.029122,-0.435924,-0.013004,-0.020709,1.033884,-0.033798,0.353010,0.525906,0.453648,0.424850
1741,1998,-0.106021,-0.088136,-0.025128,-0.024919,-0.035307,-0.242062,-0.115127,-0.427167,-0.093478,...,-0.697061,1.159923,-0.806402,-0.809586,1.033912,-0.252968,-0.728777,-1.008194,-0.453289,-0.757926
1742,1999,-0.124002,-0.105158,-0.025185,-0.024971,-0.068508,-0.098127,0.316466,1.586562,-0.104839,...,-0.391407,1.480851,-0.725574,-0.727505,1.033912,-0.852857,-0.127534,-0.522571,-0.095621,-0.490813


Essentia Best Overall + openSMILE emobase

In [67]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_emobase), columns=df_deam_integrated_essentia_best_overall_opensmile_emobase.columns)

df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised = df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,-0.092381,-0.078572,-0.025137,-0.024931,-0.051278,-0.058571,-0.055109,0.361562,-0.018698,...,1.121299,1.099986,-0.339925,-0.368315,-1.495692,0.139276,1.251933,1.502738,1.245620,1.382243
1,3,0.073364,0.050761,-0.025077,-0.024890,-0.020349,1.479718,3.014248,0.583826,0.056605,...,-0.751053,-0.671540,-0.187318,-0.107478,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970
2,4,-0.119333,-0.107505,-0.025174,-0.024965,-0.083147,-0.208423,-0.075820,0.053745,-0.094185,...,0.936125,0.958616,-0.382295,-0.388247,-0.643263,0.139276,0.393597,0.647609,0.390021,0.522072
3,5,-0.163481,-0.139436,-0.025195,-0.024978,-0.094325,-0.262338,-0.148097,-0.493194,-0.110313,...,1.053116,1.048412,-0.405301,-0.395619,-1.324591,0.139276,1.301805,1.331095,1.295332,1.320642
4,7,0.721637,0.521923,-0.024526,-0.024479,0.015265,2.154544,4.094221,0.492728,0.184390,...,0.231374,0.371407,-0.344503,-0.336807,-0.465805,0.139276,0.487485,0.469588,0.483610,0.479259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,-0.154993,-0.129360,-0.025193,-0.024975,-0.100449,-0.321333,-0.138527,0.145668,-0.130138,...,-0.969152,-0.970862,1.805891,1.493076,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970
1740,1997,-0.169892,-0.137550,-0.025197,-0.024978,-0.102115,-0.346532,-0.290196,-0.205395,-0.126981,...,-1.527775,-2.071514,0.029865,-0.142352,0.713648,0.139276,-0.854889,-0.713601,-0.854482,-0.788262
1741,1998,-0.106021,-0.088136,-0.025128,-0.024919,-0.035307,-0.242062,-0.115127,-0.427167,-0.093478,...,-0.766593,-0.690273,-0.406636,-0.268314,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970
1742,1999,-0.124002,-0.105158,-0.025185,-0.024971,-0.068508,-0.098127,0.316466,1.586562,-0.104839,...,-1.049113,-1.090194,0.711628,0.318666,0.914759,0.139276,-0.854889,-0.915349,-0.854482,-0.889970


Essentia Best Overall + openSMILE GeMAPS

In [68]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_gemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_gemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised = df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,-0.092381,-0.078572,-0.025137,-0.024931,-0.051278,-0.058571,-0.055109,0.361562,-0.018698,...,0.287564,-0.262130,-0.181091,0.668531,-0.555169,-0.967327,-0.075432,0.439999,-0.440097,-0.377481
1,3,0.073364,0.050761,-0.025077,-0.024890,-0.020349,1.479718,3.014248,0.583826,0.056605,...,-1.745983,1.481523,-1.835793,0.496337,-0.932362,0.263018,-0.389877,-0.474327,0.570241,0.450156
2,4,-0.119333,-0.107505,-0.025174,-0.024965,-0.083147,-0.208423,-0.075820,0.053745,-0.094185,...,0.025746,-0.164946,-0.758607,0.719491,0.005327,0.196521,-0.381856,-0.512572,-0.174826,-0.303339
3,5,-0.163481,-0.139436,-0.025195,-0.024978,-0.094325,-0.262338,-0.148097,-0.493194,-0.110313,...,0.296808,-0.540060,1.453477,-1.260938,-0.603349,-1.000287,-0.015498,0.340968,-0.418676,-0.300240
4,7,0.721637,0.521923,-0.024526,-0.024479,0.015265,2.154544,4.094221,0.492728,0.184390,...,-2.342634,1.977728,-0.823529,0.440421,1.121683,-0.917488,-0.139595,0.293586,-0.372262,-0.240131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,-0.154993,-0.129360,-0.025193,-0.024975,-0.100449,-0.321333,-0.138527,0.145668,-0.130138,...,0.343948,-0.753080,0.230580,-0.884061,1.768270,-0.434687,-0.329498,-0.316919,-0.508329,-0.462869
1740,1997,-0.169892,-0.137550,-0.025197,-0.024978,-0.102115,-0.346532,-0.290196,-0.205395,-0.126981,...,0.222617,-0.444289,0.247618,-0.770541,1.076880,-0.550845,-0.310073,-0.300887,-0.344771,-0.226950
1741,1998,-0.106021,-0.088136,-0.025128,-0.024919,-0.035307,-0.242062,-0.115127,-0.427167,-0.093478,...,0.374784,-0.346391,-0.400532,-0.131125,1.027496,0.996925,-0.404940,-0.493444,-0.080049,-0.197836
1742,1999,-0.124002,-0.105158,-0.025185,-0.024971,-0.068508,-0.098127,0.316466,1.586562,-0.104839,...,0.146768,-0.269664,0.058214,-0.166695,0.138567,0.629971,-0.396049,-0.541690,-0.219187,-0.239845


Essentia Best Overall + openSMILE eGeMAPS

In [69]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_egemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_egemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised = df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,-0.092381,-0.078572,-0.025137,-0.024931,-0.051278,-0.058571,-0.055109,0.361562,-0.018698,...,-0.181091,0.668531,-0.062375,-0.555169,-0.967327,-0.075432,0.439999,-0.440097,-0.377481,-0.057938
1,3,0.073364,0.050761,-0.025077,-0.024890,-0.020349,1.479718,3.014248,0.583826,0.056605,...,-1.835793,0.496337,0.172590,-0.932362,0.263018,-0.389877,-0.474327,0.570241,0.450156,0.340489
2,4,-0.119333,-0.107505,-0.025174,-0.024965,-0.083147,-0.208423,-0.075820,0.053745,-0.094185,...,-0.758607,0.719491,1.269721,0.005327,0.196521,-0.381856,-0.512572,-0.174826,-0.303339,1.245411
3,5,-0.163481,-0.139436,-0.025195,-0.024978,-0.094325,-0.262338,-0.148097,-0.493194,-0.110313,...,1.453477,-1.260938,0.286866,-0.603349,-1.000287,-0.015498,0.340968,-0.418676,-0.300240,0.719980
4,7,0.721637,0.521923,-0.024526,-0.024479,0.015265,2.154544,4.094221,0.492728,0.184390,...,-0.823529,0.440421,1.299868,1.121683,-0.917488,-0.139595,0.293586,-0.372262,-0.240131,1.938993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,-0.154993,-0.129360,-0.025193,-0.024975,-0.100449,-0.321333,-0.138527,0.145668,-0.130138,...,0.230580,-0.884061,0.419708,1.768270,-0.434687,-0.329498,-0.316919,-0.508329,-0.462869,0.346327
1740,1997,-0.169892,-0.137550,-0.025197,-0.024978,-0.102115,-0.346532,-0.290196,-0.205395,-0.126981,...,0.247618,-0.770541,-0.263554,1.076880,-0.550845,-0.310073,-0.300887,-0.344771,-0.226950,-0.749313
1741,1998,-0.106021,-0.088136,-0.025128,-0.024919,-0.035307,-0.242062,-0.115127,-0.427167,-0.093478,...,-0.400532,-0.131125,-0.070865,1.027496,0.996925,-0.404940,-0.493444,-0.080049,-0.197836,-0.015104
1742,1999,-0.124002,-0.105158,-0.025185,-0.024971,-0.068508,-0.098127,0.316466,1.586562,-0.104839,...,0.058214,-0.166695,-0.045963,0.138567,0.629971,-0.396049,-0.541690,-0.219187,-0.239845,-0.381972


### Data Normalisation

In [70]:
scaler = MinMaxScaler()

Essentia All + openSMILE ComParE2016

In [71]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_compare2016_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_compare2016), columns=df_deam_integrated_essentia_all_opensmile_compare2016.columns)

df_deam_integrated_essentia_all_opensmile_compare2016_normalised = df_deam_integrated_essentia_all_opensmile_compare2016_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_compare2016_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_compare2016_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_all_opensmile_compare2016_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,0.972741,0.322732,0.345839,0.104501,0.145450,0.704285,0.376120,0.313025,0.154835,...,0.190623,0.472256,0.277173,0.277962,1.000000e+00,0.714793,0.416608,0.221707,0.413789,0.219297
1,3,0.583679,0.205943,0.227717,0.096043,0.121328,0.970572,0.702144,0.582300,0.362288,...,0.146844,0.378009,0.212306,0.213589,5.000000e-08,0.559880,0.287637,0.164190,0.286191,0.185929
2,4,0.990960,0.375110,0.387854,0.150212,0.190855,0.658252,0.336314,0.283905,0.164182,...,0.242524,0.413426,0.290800,0.291690,1.000000e+00,0.668161,0.390070,0.235743,0.365798,0.184866
3,5,0.914417,0.356523,0.363551,0.122697,0.149895,0.858083,0.392704,0.305499,0.277203,...,0.210726,0.370086,0.266795,0.267355,9.999982e-01,0.603995,0.344762,0.204850,0.319674,0.197656
4,7,0.991187,0.256914,0.255848,0.139336,0.149169,0.824768,0.660008,0.574646,0.183156,...,0.184160,0.531845,0.184967,0.186280,2.750000e-07,0.601637,0.286155,0.131960,0.277696,0.132552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.996544,0.532901,0.553871,0.245012,0.263674,0.717103,0.318415,0.236341,0.243114,...,0.171132,0.593025,0.252964,0.253482,9.999967e-01,0.550563,0.368041,0.174927,0.388766,0.173892
1740,1997,0.998184,0.351222,0.382376,0.123357,0.160645,0.605959,0.307426,0.253574,0.218863,...,0.273023,0.462493,0.303003,0.303331,9.999858e-01,0.550175,0.425417,0.267852,0.431409,0.257782
1741,1998,0.911413,0.423552,0.429467,0.174821,0.189958,0.795025,0.269332,0.218253,0.095981,...,0.181711,0.684323,0.226191,0.226924,9.999999e-01,0.522157,0.324763,0.155054,0.345770,0.171652
1742,1999,0.974887,0.395791,0.436942,0.118189,0.156440,0.638507,0.285916,0.238424,0.279791,...,0.223496,0.728933,0.234016,0.234874,1.000000e+00,0.445470,0.380705,0.190761,0.379543,0.191104


Essentia All + openSMILE emobase

In [72]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_emobase_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_emobase), columns=df_deam_integrated_essentia_all_opensmile_emobase.columns)

df_deam_integrated_essentia_all_opensmile_emobase_normalised = df_deam_integrated_essentia_all_opensmile_emobase_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_emobase_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_emobase_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_all_opensmile_emobase_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,0.972741,0.322732,0.345839,0.104501,0.145450,0.704285,0.376120,0.313025,0.154835,...,0.427306,0.633121,0.052970,0.008991,0.546204,0.931732,0.403270,0.453796,0.401456,0.428889
1,3,0.583679,0.205943,0.227717,0.096043,0.121328,0.970572,0.702144,0.582300,0.362288,...,0.141595,0.340520,0.068615,0.031988,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000
2,4,0.990960,0.375110,0.387854,0.150212,0.190855,0.658252,0.336314,0.283905,0.164182,...,0.399049,0.609771,0.048627,0.007233,0.706684,0.931732,0.238975,0.293316,0.237900,0.266529
3,5,0.914417,0.356523,0.363551,0.122697,0.149895,0.858083,0.392704,0.305499,0.277203,...,0.416902,0.624603,0.046268,0.006583,0.578416,0.931732,0.412816,0.421584,0.410959,0.417262
4,7,0.991187,0.256914,0.255848,0.139336,0.149169,0.824768,0.660008,0.574646,0.183156,...,0.291508,0.512783,0.052501,0.011769,0.740093,0.931732,0.256946,0.259907,0.255790,0.258448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.996544,0.532901,0.553871,0.245012,0.263674,0.717103,0.318415,0.236341,0.243114,...,0.108314,0.291082,0.272948,0.173102,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000
1740,1997,0.998184,0.351222,0.382376,0.123357,0.160645,0.605959,0.307426,0.253574,0.218863,...,0.023071,0.109289,0.090879,0.028913,0.962139,0.931732,0.000000,0.037861,0.000000,0.019198
1741,1998,0.911413,0.423552,0.429467,0.174821,0.189958,0.795025,0.269332,0.218253,0.095981,...,0.139224,0.337426,0.046131,0.017807,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000
1742,1999,0.974887,0.395791,0.436942,0.118189,0.156440,0.638507,0.285916,0.238424,0.279791,...,0.096113,0.271372,0.160770,0.069559,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000


Essentia All + openSMILE GeMAPs

In [73]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_gemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_gemaps), columns=df_deam_integrated_essentia_all_opensmile_gemaps.columns)

df_deam_integrated_essentia_all_opensmile_gemaps_normalised = df_deam_integrated_essentia_all_opensmile_gemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_gemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_gemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_gemaps_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,0.972741,0.322732,0.345839,0.104501,0.145450,0.704285,0.376120,0.313025,0.154835,...,0.698817,0.312126,0.351559,0.600638,0.264131,0.049256,0.069750,0.164484,0.027513,0.014235
1,3,0.583679,0.205943,0.227717,0.096043,0.121328,0.970572,0.702144,0.582300,0.362288,...,0.425407,0.569125,0.126664,0.581319,0.206415,0.329843,0.008219,0.024446,0.077423,0.056789
2,4,0.990960,0.375110,0.387854,0.150212,0.190855,0.658252,0.336314,0.283905,0.164182,...,0.663616,0.326450,0.273067,0.606355,0.349895,0.314678,0.009789,0.018589,0.040617,0.018047
3,5,0.914417,0.356523,0.363551,0.122697,0.149895,0.858083,0.392704,0.305499,0.277203,...,0.700060,0.271162,0.573717,0.384170,0.256759,0.041739,0.081478,0.149317,0.028571,0.018206
4,7,0.991187,0.256914,0.255848,0.139336,0.149169,0.824768,0.660008,0.574646,0.183156,...,0.345188,0.642261,0.264243,0.575046,0.520714,0.060622,0.057195,0.142060,0.030864,0.021297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.996544,0.532901,0.553871,0.245012,0.263674,0.717103,0.318415,0.236341,0.243114,...,0.706398,0.239765,0.407510,0.426452,0.619651,0.170727,0.020034,0.048555,0.024143,0.009844
1740,1997,0.998184,0.351222,0.382376,0.123357,0.160645,0.605959,0.307426,0.253574,0.218863,...,0.690085,0.285278,0.409826,0.439188,0.513858,0.144237,0.023835,0.051010,0.032222,0.021974
1741,1998,0.911413,0.423552,0.429467,0.174821,0.189958,0.795025,0.269332,0.218253,0.095981,...,0.710544,0.299707,0.321734,0.510924,0.506302,0.497215,0.005271,0.021518,0.045299,0.023471
1742,1999,0.974887,0.395791,0.436942,0.118189,0.156440,0.638507,0.285916,0.238424,0.279791,...,0.679887,0.311016,0.384083,0.506933,0.370283,0.413529,0.007011,0.014129,0.038426,0.021311


Essentia All + openSMILE eGeMAPS

In [74]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_egemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_egemaps), columns=df_deam_integrated_essentia_all_opensmile_egemaps.columns)

df_deam_integrated_essentia_all_opensmile_egemaps_normalised = df_deam_integrated_essentia_all_opensmile_egemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_egemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_egemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_egemaps_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,0.972741,0.322732,0.345839,0.104501,0.145450,0.704285,0.376120,0.313025,0.154835,...,0.351559,0.600638,0.181705,0.264131,0.049256,0.069750,0.164484,0.027513,0.014235,0.594429
1,3,0.583679,0.205943,0.227717,0.096043,0.121328,0.970572,0.702144,0.582300,0.362288,...,0.126664,0.581319,0.212856,0.206415,0.329843,0.008219,0.024446,0.077423,0.056789,0.639348
2,4,0.990960,0.375110,0.387854,0.150212,0.190855,0.658252,0.336314,0.283905,0.164182,...,0.273067,0.606355,0.358312,0.349895,0.314678,0.009789,0.018589,0.040617,0.018047,0.741370
3,5,0.914417,0.356523,0.363551,0.122697,0.149895,0.858083,0.392704,0.305499,0.277203,...,0.573717,0.384170,0.228007,0.256759,0.041739,0.081478,0.149317,0.028571,0.018206,0.682132
4,7,0.991187,0.256914,0.255848,0.139336,0.149169,0.824768,0.660008,0.574646,0.183156,...,0.264243,0.575046,0.362308,0.520714,0.060622,0.057195,0.142060,0.030864,0.021297,0.819566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.996544,0.532901,0.553871,0.245012,0.263674,0.717103,0.318415,0.236341,0.243114,...,0.407510,0.426452,0.245619,0.619651,0.170727,0.020034,0.048555,0.024143,0.009844,0.640006
1740,1997,0.998184,0.351222,0.382376,0.123357,0.160645,0.605959,0.307426,0.253574,0.218863,...,0.409826,0.439188,0.155033,0.513858,0.144237,0.023835,0.051010,0.032222,0.021974,0.516482
1741,1998,0.911413,0.423552,0.429467,0.174821,0.189958,0.795025,0.269332,0.218253,0.095981,...,0.321734,0.510924,0.180579,0.506302,0.497215,0.005271,0.021518,0.045299,0.023471,0.599258
1742,1999,0.974887,0.395791,0.436942,0.118189,0.156440,0.638507,0.285916,0.238424,0.279791,...,0.384083,0.506933,0.183881,0.370283,0.413529,0.007011,0.014129,0.038426,0.021311,0.557897


Essentia Best Overall + openSMILE ComParE2016

In [75]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_compare2016), columns=df_deam_integrated_essentia_best_overall_opensmile_compare2016.columns)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised = df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2,0.005020,0.004055,1.866850e-06,1.524876e-06,0.001798,0.023745,0.027549,0.131211,0.003891,...,0.190623,0.472256,0.277173,0.277962,1.000000e+00,0.714793,0.416608,0.221707,0.413789,0.219297
1,3,0.009253,0.007287,3.311122e-06,2.524975e-06,0.002593,0.082083,0.140294,0.151787,0.005776,...,0.146844,0.378009,0.212306,0.213589,5.000000e-08,0.559880,0.287637,0.164190,0.286191,0.185929
2,4,0.004331,0.003332,9.901372e-07,7.146511e-07,0.000979,0.018062,0.026788,0.102714,0.002002,...,0.242524,0.413426,0.290800,0.291690,1.000000e+00,0.668161,0.390070,0.235743,0.365798,0.184866
3,5,0.003204,0.002534,4.937064e-07,4.159568e-07,0.000692,0.016018,0.024134,0.052080,0.001598,...,0.210726,0.370086,0.266795,0.267355,9.999982e-01,0.603995,0.344762,0.204850,0.319674,0.197656
4,7,0.025812,0.019064,1.650024e-05,1.234842e-05,0.003508,0.107675,0.179964,0.143354,0.008974,...,0.184160,0.531845,0.184967,0.186280,2.750000e-07,0.601637,0.286155,0.131960,0.277696,0.132552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.003420,0.002785,5.455507e-07,4.706971e-07,0.000535,0.013781,0.024485,0.111224,0.001102,...,0.171132,0.593025,0.252964,0.253482,9.999967e-01,0.550563,0.368041,0.174927,0.388766,0.173892
1740,1997,0.003040,0.002581,4.429354e-07,4.051469e-07,0.000492,0.012825,0.018914,0.078724,0.001181,...,0.273023,0.462493,0.303003,0.303331,9.999858e-01,0.550175,0.425417,0.267852,0.431409,0.257782
1741,1998,0.004671,0.003816,2.087614e-06,1.831211e-06,0.002208,0.016787,0.025345,0.058193,0.002020,...,0.181711,0.684323,0.226191,0.226924,9.999999e-01,0.522157,0.324763,0.155054,0.345770,0.171652
1742,1999,0.004212,0.003390,7.172558e-07,5.847591e-07,0.001355,0.022245,0.041198,0.244618,0.001735,...,0.223496,0.728933,0.234016,0.234874,1.000000e+00,0.445470,0.380705,0.190761,0.379543,0.191104


Essentia Best Overall + openSMILE emobase

In [76]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_emobase), columns=df_deam_integrated_essentia_best_overall_opensmile_emobase.columns)

df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised = df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,2,0.005020,0.004055,1.866850e-06,1.524876e-06,0.001798,0.023745,0.027549,0.131211,0.003891,...,0.427306,0.633121,0.052970,0.008991,0.546204,0.931732,0.403270,0.453796,0.401456,0.428889
1,3,0.009253,0.007287,3.311122e-06,2.524975e-06,0.002593,0.082083,0.140294,0.151787,0.005776,...,0.141595,0.340520,0.068615,0.031988,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000
2,4,0.004331,0.003332,9.901372e-07,7.146511e-07,0.000979,0.018062,0.026788,0.102714,0.002002,...,0.399049,0.609771,0.048627,0.007233,0.706684,0.931732,0.238975,0.293316,0.237900,0.266529
3,5,0.003204,0.002534,4.937064e-07,4.159568e-07,0.000692,0.016018,0.024134,0.052080,0.001598,...,0.416902,0.624603,0.046268,0.006583,0.578416,0.931732,0.412816,0.421584,0.410959,0.417262
4,7,0.025812,0.019064,1.650024e-05,1.234842e-05,0.003508,0.107675,0.179964,0.143354,0.008974,...,0.291508,0.512783,0.052501,0.011769,0.740093,0.931732,0.256946,0.259907,0.255790,0.258448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.003420,0.002785,5.455507e-07,4.706971e-07,0.000535,0.013781,0.024485,0.111224,0.001102,...,0.108314,0.291082,0.272948,0.173102,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000
1740,1997,0.003040,0.002581,4.429354e-07,4.051469e-07,0.000492,0.012825,0.018914,0.078724,0.001181,...,0.023071,0.109289,0.090879,0.028913,0.962139,0.931732,0.000000,0.037861,0.000000,0.019198
1741,1998,0.004671,0.003816,2.087614e-06,1.831211e-06,0.002208,0.016787,0.025345,0.058193,0.002020,...,0.139224,0.337426,0.046131,0.017807,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000
1742,1999,0.004212,0.003390,7.172558e-07,5.847591e-07,0.001355,0.022245,0.041198,0.244618,0.001735,...,0.096113,0.271372,0.160770,0.069559,1.000000,0.931732,0.000000,0.000000,0.000000,0.000000


Essentia Best Overall + openSMILE GeMAPS

In [77]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_gemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_gemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised = df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,0.005020,0.004055,1.866850e-06,1.524876e-06,0.001798,0.023745,0.027549,0.131211,0.003891,...,0.698817,0.312126,0.351559,0.600638,0.264131,0.049256,0.069750,0.164484,0.027513,0.014235
1,3,0.009253,0.007287,3.311122e-06,2.524975e-06,0.002593,0.082083,0.140294,0.151787,0.005776,...,0.425407,0.569125,0.126664,0.581319,0.206415,0.329843,0.008219,0.024446,0.077423,0.056789
2,4,0.004331,0.003332,9.901372e-07,7.146511e-07,0.000979,0.018062,0.026788,0.102714,0.002002,...,0.663616,0.326450,0.273067,0.606355,0.349895,0.314678,0.009789,0.018589,0.040617,0.018047
3,5,0.003204,0.002534,4.937064e-07,4.159568e-07,0.000692,0.016018,0.024134,0.052080,0.001598,...,0.700060,0.271162,0.573717,0.384170,0.256759,0.041739,0.081478,0.149317,0.028571,0.018206
4,7,0.025812,0.019064,1.650024e-05,1.234842e-05,0.003508,0.107675,0.179964,0.143354,0.008974,...,0.345188,0.642261,0.264243,0.575046,0.520714,0.060622,0.057195,0.142060,0.030864,0.021297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.003420,0.002785,5.455507e-07,4.706971e-07,0.000535,0.013781,0.024485,0.111224,0.001102,...,0.706398,0.239765,0.407510,0.426452,0.619651,0.170727,0.020034,0.048555,0.024143,0.009844
1740,1997,0.003040,0.002581,4.429354e-07,4.051469e-07,0.000492,0.012825,0.018914,0.078724,0.001181,...,0.690085,0.285278,0.409826,0.439188,0.513858,0.144237,0.023835,0.051010,0.032222,0.021974
1741,1998,0.004671,0.003816,2.087614e-06,1.831211e-06,0.002208,0.016787,0.025345,0.058193,0.002020,...,0.710544,0.299707,0.321734,0.510924,0.506302,0.497215,0.005271,0.021518,0.045299,0.023471
1742,1999,0.004212,0.003390,7.172558e-07,5.847591e-07,0.001355,0.022245,0.041198,0.244618,0.001735,...,0.679887,0.311016,0.384083,0.506933,0.370283,0.413529,0.007011,0.014129,0.038426,0.021311


Essentia Best Overall + openSMILE eGeMAPS

In [78]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_egemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_egemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised = df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,2,0.005020,0.004055,1.866850e-06,1.524876e-06,0.001798,0.023745,0.027549,0.131211,0.003891,...,0.351559,0.600638,0.181705,0.264131,0.049256,0.069750,0.164484,0.027513,0.014235,0.594429
1,3,0.009253,0.007287,3.311122e-06,2.524975e-06,0.002593,0.082083,0.140294,0.151787,0.005776,...,0.126664,0.581319,0.212856,0.206415,0.329843,0.008219,0.024446,0.077423,0.056789,0.639348
2,4,0.004331,0.003332,9.901372e-07,7.146511e-07,0.000979,0.018062,0.026788,0.102714,0.002002,...,0.273067,0.606355,0.358312,0.349895,0.314678,0.009789,0.018589,0.040617,0.018047,0.741370
3,5,0.003204,0.002534,4.937064e-07,4.159568e-07,0.000692,0.016018,0.024134,0.052080,0.001598,...,0.573717,0.384170,0.228007,0.256759,0.041739,0.081478,0.149317,0.028571,0.018206,0.682132
4,7,0.025812,0.019064,1.650024e-05,1.234842e-05,0.003508,0.107675,0.179964,0.143354,0.008974,...,0.264243,0.575046,0.362308,0.520714,0.060622,0.057195,0.142060,0.030864,0.021297,0.819566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.003420,0.002785,5.455507e-07,4.706971e-07,0.000535,0.013781,0.024485,0.111224,0.001102,...,0.407510,0.426452,0.245619,0.619651,0.170727,0.020034,0.048555,0.024143,0.009844,0.640006
1740,1997,0.003040,0.002581,4.429354e-07,4.051469e-07,0.000492,0.012825,0.018914,0.078724,0.001181,...,0.409826,0.439188,0.155033,0.513858,0.144237,0.023835,0.051010,0.032222,0.021974,0.516482
1741,1998,0.004671,0.003816,2.087614e-06,1.831211e-06,0.002208,0.016787,0.025345,0.058193,0.002020,...,0.321734,0.510924,0.180579,0.506302,0.497215,0.005271,0.021518,0.045299,0.023471,0.599258
1742,1999,0.004212,0.003390,7.172558e-07,5.847591e-07,0.001355,0.022245,0.041198,0.244618,0.001735,...,0.384083,0.506933,0.183881,0.370283,0.413529,0.007011,0.014129,0.038426,0.021311,0.557897


## Filter out only the mean features

In [79]:
df_essentia_best_overall_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


In [80]:
col_names = df_essentia_best_overall_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

['song_id', 'lowlevel.melbands_kurtosis.mean', 'lowlevel.melbands_skewness.mean', 'lowlevel.spectral_energy.mean', 'tonal.chords_strength.mean', 'tonal.hpcp_entropy.mean', 'tonal.key_edma.strength', 'tonal.key_temperley.strength', 'rhythm.beats_loudness_band_ratio.mean_0', 'rhythm.beats_loudness_band_ratio.mean_1', 'rhythm.beats_loudness_band_ratio.mean_2', 'rhythm.beats_loudness_band_ratio.mean_3', 'rhythm.beats_loudness_band_ratio.mean_4', 'rhythm.beats_loudness_band_ratio.mean_5', 'tonal.chords_histogram_0', 'tonal.chords_histogram_1', 'tonal.chords_histogram_2', 'tonal.chords_histogram_3', 'tonal.chords_histogram_4', 'tonal.chords_histogram_5', 'tonal.chords_histogram_6', 'tonal.chords_histogram_7', 'tonal.chords_histogram_8', 'tonal.chords_histogram_9', 'tonal.chords_histogram_10', 'tonal.chords_histogram_11', 'tonal.chords_histogram_12', 'tonal.chords_histogram_13', 'tonal.chords_histogram_14', 'tonal.chords_histogram_15', 'tonal.chords_histogram_16', 'tonal.chords_histogram_17',

In [81]:
df_essentia_best_overall_features_mean = df_essentia_best_overall_features[feature_mean_cols]
df_essentia_best_overall_features_mean

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,tonal.key_temperley.strength,rhythm.beats_loudness_band_ratio.mean_0,rhythm.beats_loudness_band_ratio.mean_1,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,21.801605,3.360947,0.033098,0.488094,2.154713,0.654566,0.632045,0.565605,0.220067,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,76.052628,5.752009,0.112371,0.491757,1.938500,0.691578,0.691432,0.884698,0.097718,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,16.516722,3.156479,0.048238,0.582855,1.964482,0.631611,0.616269,0.669513,0.172599,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,14.615296,2.535792,0.026126,0.579267,1.571803,0.852302,0.847586,0.268164,0.353358,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,99.851807,5.768692,0.105269,0.414062,2.082458,0.528712,0.518792,0.915227,0.065957,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,12.534721,2.630574,0.044680,0.517544,2.142346,0.759683,0.756458,0.233401,0.533388,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,11.646033,2.686466,0.042499,0.515905,2.018864,0.600971,0.620554,0.615935,0.228900,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,15.330372,2.877960,0.033532,0.514480,2.212357,0.724135,0.737769,0.618689,0.147210,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,20.406567,3.374909,0.032682,0.439860,2.140274,0.685962,0.707858,0.544488,0.240996,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


In [82]:
df_essentia_best_overall_features_mean.to_csv(get_deam_path('processed/features/essentia_best_overall_features_mean.csv'))

Normalisation

In [83]:
scaler = MinMaxScaler()

In [84]:
# Fit and transform the selected columns
df_essentia_best_overall_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_overall_features_mean), columns=df_essentia_best_overall_features_mean.columns)

df_essentia_best_overall_features_mean_normalised = df_essentia_best_overall_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_overall_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_overall_features_mean_normalised.to_csv(get_deam_path('processed/features/normalised_essentia_best_overall_features_mean.csv'))

df_essentia_best_overall_features_mean_normalised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,tonal.key_temperley.strength,rhythm.beats_loudness_band_ratio.mean_0,rhythm.beats_loudness_band_ratio.mean_1,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.023745,0.224209,0.054855,0.392382,0.631265,0.625832,0.593381,0.579839,0.267674,...,0.023256,0.010375,0.064544,0.026229,0.046262,0.005850,0.00000,0.083526,0.035887,0.012672
1,3,0.082083,0.348482,0.211289,0.402792,0.543954,0.677312,0.677951,0.907102,0.118704,...,0.000000,0.000000,0.038179,0.000000,0.000000,0.000000,0.00000,0.000000,0.023489,0.035942
2,4,0.018062,0.213582,0.084731,0.661722,0.554446,0.593904,0.570916,0.686407,0.209877,...,0.000000,0.001484,0.002937,0.000000,0.049618,0.000000,0.00000,0.000000,0.000000,0.001057
3,5,0.016018,0.181322,0.041096,0.651524,0.395875,0.900862,0.900318,0.274781,0.429967,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.049780,0.00000,0.005017,0.127117,0.022199
4,7,0.107675,0.349349,0.197274,0.181959,0.602087,0.450784,0.432106,0.938413,0.080032,...,0.000000,0.000000,0.000000,0.068694,0.000000,0.162351,0.18304,0.125289,0.000000,0.015840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.013781,0.186248,0.077709,0.476088,0.626271,0.772039,0.770549,0.239129,0.649168,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009682,0.021164
1740,1997,0.012825,0.189153,0.073405,0.471430,0.576407,0.551288,0.577018,0.631457,0.278428,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.021985,0.00000,0.140613,0.222683,0.275132
1741,1998,0.016787,0.199106,0.055710,0.467379,0.654543,0.722595,0.743935,0.634282,0.178965,...,0.000000,0.017823,0.000000,0.426784,0.000000,0.353219,0.00000,0.000000,0.038728,0.001058
1742,1999,0.022245,0.224935,0.054033,0.255286,0.625434,0.669501,0.701341,0.558181,0.293156,...,0.068617,0.016337,0.336617,0.005006,0.185430,0.054229,0.00000,0.468710,0.100968,0.176720


Standardisation

In [85]:
scaler = StandardScaler()

In [86]:
# Fit and transform the selected columns
df_essentia_best_overall_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_overall_features_mean), columns=df_essentia_best_overall_features_mean.columns)

df_essentia_best_overall_features_mean_standardised = df_essentia_best_overall_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_overall_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_overall_features_mean_standardised.to_csv(get_deam_path('processed/features/standardised_essentia_best_overall_features_mean.csv'))

df_essentia_best_overall_features_mean_standardised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,tonal.key_temperley.strength,rhythm.beats_loudness_band_ratio.mean_0,rhythm.beats_loudness_band_ratio.mean_1,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,-0.058571,0.380991,-0.069127,-0.452484,0.809781,-0.085112,-0.288833,0.569221,-0.098158,...,0.231912,-0.169071,0.693755,-0.215917,0.174944,-0.382824,-0.398789,0.323740,-0.466431,-0.596685
1,3,1.479718,2.085477,3.025151,-0.386129,0.202518,0.233522,0.222782,2.000293,-0.968809,...,-0.220673,-0.312868,0.297680,-0.477019,-0.368661,-0.434446,-0.398789,-0.487958,-0.563641,-0.386679
2,4,-0.208423,0.235235,0.521816,1.264373,0.275493,-0.282729,-0.424737,1.035227,-0.435947,...,-0.220673,-0.292305,-0.231753,-0.477019,0.214373,-0.434446,-0.398789,-0.487958,-0.747808,-0.701512
3,5,-0.262338,-0.207226,-0.341279,1.199370,-0.827397,1.617190,1.568023,-0.764753,0.850362,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,0.004792,-0.398789,-0.439206,0.248861,-0.510704
4,7,2.154544,2.097370,2.747921,-1.793781,0.606843,-1.168573,-1.264489,2.137213,-1.194825,...,-0.220673,-0.312868,-0.275872,0.206820,-0.368661,0.998061,1.904098,0.729588,-0.747808,-0.568093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,-0.321333,-0.139660,0.382933,0.081083,0.775046,0.819838,0.782967,-0.920655,2.131480,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,-0.434446,-0.398789,-0.487958,-0.671897,-0.520047
1740,1997,-0.346532,-0.099817,0.297799,0.051395,0.428232,-0.546505,-0.387822,0.794940,-0.035303,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,-0.240465,-0.398789,0.878506,0.998161,1.772008
1741,1998,-0.242062,0.036691,-0.052204,0.025573,0.971682,0.513806,0.621963,0.807291,-0.616616,...,-0.220673,-0.065850,-0.275872,3.771523,-0.368661,2.682184,-0.398789,-0.487958,-0.444162,-0.701502
1742,1999,-0.098127,0.390945,-0.085379,-1.326377,0.769227,0.185179,0.364287,0.474512,0.050776,...,1.114688,-0.086435,4.781004,-0.427183,1.810241,0.044041,-0.398789,4.066921,0.043842,0.883837


In [87]:
df_essentia_best_arousal_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.dmean,lowlevel.barkbands_spread.dmean2,lowlevel.barkbands_spread.dvar,lowlevel.barkbands_spread.dvar2,lowlevel.barkbands_spread.max,lowlevel.barkbands_spread.mean,lowlevel.barkbands_spread.median,lowlevel.barkbands_spread.min,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.960248,4.550666,7.235502,37.865948,95.265396,94.332970,15.924681,14.074156,1.380657,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,0.577547,1.150496,1.781423,8.920459,20.415138,65.661095,4.013522,2.604582,0.105044,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,0.978169,4.410288,6.733066,22.211205,48.142967,82.455566,15.952667,14.231352,0.854718,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,0.902877,2.369953,3.454134,19.432764,34.974041,86.441460,9.326209,7.617138,0.611234,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,0.978392,2.122688,3.430682,29.966152,82.799446,91.631195,3.206823,1.772354,0.222541,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,3.652898,5.749208,10.690978,24.899950,40.925282,13.047432,11.716851,1.945637,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,0.985275,3.603367,5.866543,12.558491,31.335993,49.654797,15.037008,14.095716,1.787386,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,0.899922,6.775062,10.153251,75.743263,166.500168,106.074104,22.843222,17.990805,2.192569,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,0.962358,3.589267,5.683703,14.578965,35.868526,69.180779,13.325944,12.539411,2.097204,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


In [88]:
col_names = df_essentia_best_arousal_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

['song_id', 'lowlevel.average_loudness', 'lowlevel.barkbands_spread.mean', 'lowlevel.melbands_crest.mean', 'lowlevel.melbands_flatness_db.mean', 'lowlevel.melbands_kurtosis.mean', 'lowlevel.melbands_skewness.mean', 'lowlevel.melbands_spread.mean', 'lowlevel.spectral_energy.mean', 'lowlevel.spectral_entropy.mean', 'lowlevel.spectral_flux.mean', 'lowlevel.spectral_kurtosis.mean', 'lowlevel.spectral_rolloff.mean', 'lowlevel.spectral_skewness.mean', 'rhythm.bpm_histogram_first_peak_bpm', 'rhythm.bpm_histogram_first_peak_weight', 'rhythm.bpm_histogram_second_peak_bpm', 'rhythm.bpm_histogram_second_peak_spread', 'rhythm.bpm_histogram_second_peak_weight', 'rhythm.danceability', 'rhythm.onset_rate', 'tonal.chords_strength.mean', 'tonal.hpcp_entropy.mean', 'tonal.key_edma.strength', 'tonal.key_temperley.strength', 'rhythm.beats_loudness_band_ratio.mean_0', 'rhythm.beats_loudness_band_ratio.mean_1', 'rhythm.beats_loudness_band_ratio.mean_2', 'rhythm.beats_loudness_band_ratio.mean_3', 'rhythm.bea

In [89]:
df_essentia_best_arousal_features_mean = df_essentia_best_arousal_features[feature_mean_cols]
df_essentia_best_arousal_features_mean

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.mean,lowlevel.melbands_crest.mean,lowlevel.melbands_flatness_db.mean,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.melbands_spread.mean,lowlevel.spectral_energy.mean,lowlevel.spectral_entropy.mean,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.960248,15.924681,17.874039,0.250065,21.801605,3.360947,15.213296,0.033098,7.259169,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,0.577547,4.013522,19.723104,0.415665,76.052628,5.752009,3.132341,0.112371,6.568457,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,0.978169,15.952667,16.846857,0.264107,16.516722,3.156479,16.124945,0.048238,7.122410,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,0.902877,9.326209,14.198059,0.278122,14.615296,2.535792,11.025191,0.026126,6.673684,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,0.978392,3.206823,20.270206,0.418586,99.851807,5.768692,2.417604,0.105269,6.504210,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.983662,13.047432,11.820193,0.271790,12.534721,2.630574,13.541412,0.044680,7.088806,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,0.985275,15.037008,14.395770,0.371013,11.646033,2.686466,16.283945,0.042499,6.981311,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,0.899922,22.843222,13.350397,0.199808,15.330372,2.877960,21.008831,0.033532,7.702393,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,0.962358,13.325944,12.950244,0.238266,20.406567,3.374909,12.168412,0.032682,7.371930,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


In [90]:
df_essentia_best_arousal_features_mean.to_csv(get_deam_path('processed/features/essentia_best_arousal_features_mean.csv'))

Normalisation

In [91]:
scaler = MinMaxScaler()

In [92]:
# Fit and transform the selected columns
df_essentia_best_arousal_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_arousal_features_mean), columns=df_essentia_best_arousal_features_mean.columns)

df_essentia_best_arousal_features_mean_normalised = df_essentia_best_arousal_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_arousal_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_arousal_features_mean_normalised.to_csv(get_deam_path('processed/features/normalised_essentia_best_arousal_features_mean.csv'))

df_essentia_best_arousal_features_mean_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.mean,lowlevel.melbands_crest.mean,lowlevel.melbands_flatness_db.mean,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.melbands_spread.mean,lowlevel.spectral_energy.mean,lowlevel.spectral_entropy.mean,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.972741,0.252048,0.432744,0.208882,0.023745,0.224209,0.173672,0.054855,0.788542,...,0.023256,0.010375,0.064544,0.026229,0.046262,0.005850,0.00000,0.083526,0.035887,0.012672
1,3,0.583679,0.053319,0.513446,0.442710,0.082083,0.348482,0.026271,0.211289,0.641965,...,0.000000,0.000000,0.038179,0.000000,0.000000,0.000000,0.00000,0.000000,0.023489,0.035942
2,4,0.990960,0.252515,0.387914,0.228709,0.018062,0.213582,0.184795,0.084731,0.759520,...,0.000000,0.001484,0.002937,0.000000,0.049618,0.000000,0.00000,0.000000,0.000000,0.001057
3,5,0.914417,0.141958,0.272308,0.248498,0.016018,0.181322,0.122572,0.041096,0.664296,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.049780,0.00000,0.005017,0.127117,0.022199
4,7,0.991187,0.039860,0.537324,0.446834,0.107675,0.349349,0.017550,0.197274,0.628331,...,0.000000,0.000000,0.000000,0.068694,0.000000,0.162351,0.18304,0.125289,0.000000,0.015840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.996544,0.204043,0.168527,0.239557,0.013781,0.186248,0.153273,0.077709,0.752389,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009682,0.021164
1740,1997,0.998184,0.237238,0.280937,0.379660,0.012825,0.189153,0.186735,0.073405,0.729577,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.021985,0.00000,0.140613,0.222683,0.275132
1741,1998,0.911413,0.367479,0.235312,0.137918,0.016787,0.199106,0.244384,0.055710,0.882598,...,0.000000,0.017823,0.000000,0.426784,0.000000,0.353219,0.00000,0.000000,0.038728,0.001058
1742,1999,0.974887,0.208690,0.217848,0.192222,0.022245,0.224935,0.136521,0.054033,0.812471,...,0.068617,0.016337,0.336617,0.005006,0.185430,0.054229,0.00000,0.468710,0.100968,0.176720


Standardisation

In [93]:
scaler = StandardScaler()

In [94]:
# Fit and transform the selected columns
df_essentia_best_arousal_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_arousal_features_mean), columns=df_essentia_best_arousal_features_mean.columns)

df_essentia_best_arousal_features_mean_standardised = df_essentia_best_arousal_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_arousal_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_arousal_features_mean_standardised.to_csv(get_deam_path('processed/features/standardised_essentia_best_arousal_features_mean.csv'))

df_essentia_best_arousal_features_mean_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.mean,lowlevel.melbands_crest.mean,lowlevel.melbands_flatness_db.mean,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.melbands_spread.mean,lowlevel.spectral_energy.mean,lowlevel.spectral_entropy.mean,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.568151,0.031439,0.721071,-0.407956,-0.058571,0.380991,-0.201815,-0.069127,0.480496,...,0.231912,-0.169071,0.693755,-0.215917,0.174944,-0.382824,-0.398789,0.323740,-0.466431,-0.596685
1,3,-1.189985,-1.283440,1.333029,1.226911,1.479718,2.085477,-1.345582,3.025151,-0.472704,...,-0.220673,-0.312868,0.297680,-0.477019,-0.368661,-0.434446,-0.398789,-0.487958,-0.563641,-0.386679
2,4,0.650480,0.034529,0.381119,-0.269335,-0.208423,0.235235,-0.115504,0.521816,0.291765,...,-0.220673,-0.292305,-0.231753,-0.477019,0.214373,-0.434446,-0.398789,-0.487958,-0.747808,-0.701512
3,5,0.304588,-0.696970,-0.495515,-0.130973,-0.262338,-0.207226,-0.598325,-0.341279,-0.327488,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,0.004792,-0.398789,-0.439206,0.248861,-0.510704
4,7,0.651505,-1.372492,1.514095,1.255745,2.154544,2.097370,-1.413250,2.747921,-0.561367,...,-0.220673,-0.312868,-0.275872,0.206820,-0.368661,0.998061,1.904098,0.729588,-0.747808,-0.568093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.675713,-0.286182,-1.282483,-0.193486,-0.321333,-0.139660,-0.360101,0.382933,0.245391,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,-0.434446,-0.398789,-0.487958,-0.671897,-0.520047
1740,1997,0.683125,-0.066552,-0.430082,0.786081,-0.346532,-0.099817,-0.100451,0.297799,0.097045,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,-0.240465,-0.398789,0.878506,0.998161,1.772008
1741,1998,0.291013,0.795180,-0.776054,-0.904120,-0.242062,0.036691,0.346879,-0.052204,1.092156,...,-0.220673,-0.065850,-0.275872,3.771523,-0.368661,2.682184,-0.398789,-0.487958,-0.444162,-0.701502
1742,1999,0.577845,-0.255437,-0.908487,-0.524441,-0.098127,0.390945,-0.490090,-0.085379,0.636109,...,1.114688,-0.086435,4.781004,-0.427183,1.810241,0.044041,-0.398789,4.066921,0.043842,0.883837


In [95]:
df_essentia_best_valence_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


In [96]:
col_names = df_essentia_best_valence_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

['song_id', 'lowlevel.melbands_kurtosis.mean', 'lowlevel.melbands_skewness.mean', 'lowlevel.spectral_energy.mean', 'lowlevel.zerocrossingrate.mean', 'rhythm.beats_loudness.mean', 'rhythm.onset_rate', 'tonal.chords_strength.mean', 'tonal.hpcp_entropy.mean', 'tonal.key_edma.strength', 'tonal.key_temperley.strength', 'lowlevel.gfcc.cov_0', 'lowlevel.gfcc.cov_1', 'lowlevel.gfcc.cov_2', 'lowlevel.gfcc.cov_3', 'lowlevel.gfcc.cov_4', 'lowlevel.gfcc.cov_5', 'lowlevel.gfcc.cov_6', 'lowlevel.gfcc.cov_7', 'lowlevel.gfcc.cov_8', 'lowlevel.gfcc.cov_9', 'lowlevel.gfcc.cov_10', 'lowlevel.gfcc.cov_11', 'lowlevel.gfcc.cov_12', 'lowlevel.gfcc.cov_13', 'lowlevel.gfcc.cov_14', 'lowlevel.gfcc.cov_15', 'lowlevel.gfcc.cov_16', 'lowlevel.gfcc.cov_17', 'lowlevel.gfcc.cov_18', 'lowlevel.gfcc.cov_19', 'lowlevel.gfcc.cov_20', 'lowlevel.gfcc.cov_21', 'lowlevel.gfcc.cov_22', 'lowlevel.gfcc.cov_23', 'lowlevel.gfcc.cov_24', 'lowlevel.gfcc.cov_25', 'lowlevel.gfcc.cov_26', 'lowlevel.gfcc.cov_27', 'lowlevel.gfcc.cov_28'

In [97]:
df_essentia_best_valence_features_mean = df_essentia_best_valence_features[feature_mean_cols]
df_essentia_best_valence_features_mean

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,lowlevel.zerocrossingrate.mean,rhythm.beats_loudness.mean,rhythm.onset_rate,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,21.801605,3.360947,0.033098,0.036990,0.039522,3.149848,0.488094,2.154713,0.654566,...,1.851852,0.720165,4.526749,2.160494,1.440329,0.411523,0.000000,5.144033,2.674897,1.234568
1,3,76.052628,5.752009,0.112371,0.013975,0.167025,3.019088,0.491757,1.938500,0.691578,...,0.000000,0.000000,2.677652,0.000000,0.000000,0.000000,0.000000,0.000000,1.750772,3.501545
2,4,16.516722,3.156479,0.048238,0.037837,0.070825,4.328840,0.582855,1.964482,0.631611,...,0.000000,0.102987,0.205973,0.000000,1.544799,0.000000,0.000000,0.000000,0.000000,0.102987
3,5,14.615296,2.535792,0.026126,0.036749,0.036762,3.196682,0.579267,1.571803,0.852302,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.501545,0.000000,0.308960,9.474769,2.162719
4,7,99.851807,5.768692,0.105269,0.013364,0.130131,5.368051,0.414062,2.082458,0.528712,...,0.000000,0.000000,0.000000,5.658436,0.000000,11.419753,7.510288,7.716049,0.000000,1.543210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,12.534721,2.630574,0.044680,0.033501,0.050656,3.399102,0.517544,2.142346,0.759683,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721649,2.061856
1740,1997,11.646033,2.686466,0.042499,0.037577,0.043897,2.443799,0.515905,2.018864,0.600971,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.546392,0.000000,8.659794,16.597939,26.804123
1741,1998,15.330372,2.877960,0.033532,0.064851,0.052681,4.621002,0.514480,2.212357,0.724135,...,0.000000,1.237113,0.000000,35.154640,0.000000,24.845362,0.000000,0.000000,2.886598,0.103093
1742,1999,20.406567,3.374909,0.032682,0.036931,0.049652,4.287756,0.439860,2.140274,0.685962,...,5.463918,1.134021,23.608248,0.412371,5.773196,3.814433,0.000000,28.865980,7.525773,17.216496


In [98]:
df_essentia_best_valence_features_mean.to_csv(get_deam_path('processed/features/essentia_best_valence_features_mean.csv'))

Normalisation

In [99]:
scaler = MinMaxScaler()

In [100]:
# Fit and transform the selected columns
df_essentia_best_valence_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_valence_features_mean), columns=df_essentia_best_valence_features_mean.columns)

df_essentia_best_valence_features_mean_normalised = df_essentia_best_valence_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_valence_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_valence_features_mean_normalised.to_csv(get_deam_path('processed/features/normalised_essentia_best_valence_features_mean.csv'))

df_essentia_best_valence_features_mean_normalised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,lowlevel.zerocrossingrate.mean,rhythm.beats_loudness.mean,rhythm.onset_rate,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,0.023745,0.224209,0.054855,0.087567,0.069568,0.373016,0.392382,0.631265,0.625832,...,0.023256,0.010375,0.064544,0.026229,0.046262,0.005850,0.00000,0.083526,0.035887,0.012672
1,3,0.082083,0.348482,0.211289,0.019437,0.338074,0.357421,0.402792,0.543954,0.677312,...,0.000000,0.000000,0.038179,0.000000,0.000000,0.000000,0.00000,0.000000,0.023489,0.035942
2,4,0.018062,0.213582,0.084731,0.090074,0.135489,0.513626,0.661722,0.554446,0.593904,...,0.000000,0.001484,0.002937,0.000000,0.049618,0.000000,0.00000,0.000000,0.000000,0.001057
3,5,0.016018,0.181322,0.041096,0.086854,0.063757,0.378601,0.651524,0.395875,0.900862,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.049780,0.00000,0.005017,0.127117,0.022199
4,7,0.107675,0.349349,0.197274,0.017629,0.260381,0.637566,0.181959,0.602087,0.450784,...,0.000000,0.000000,0.000000,0.068694,0.000000,0.162351,0.18304,0.125289,0.000000,0.015840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,0.013781,0.186248,0.077709,0.077239,0.093014,0.402743,0.476088,0.626271,0.772039,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009682,0.021164
1740,1997,0.012825,0.189153,0.073405,0.089305,0.078782,0.288810,0.471430,0.576407,0.551288,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.021985,0.00000,0.140613,0.222683,0.275132
1741,1998,0.016787,0.199106,0.055710,0.170043,0.097281,0.548471,0.467379,0.654543,0.722595,...,0.000000,0.017823,0.000000,0.426784,0.000000,0.353219,0.00000,0.000000,0.038728,0.001058
1742,1999,0.022245,0.224935,0.054033,0.087392,0.090900,0.508727,0.255286,0.625434,0.669501,...,0.068617,0.016337,0.336617,0.005006,0.185430,0.054229,0.00000,0.468710,0.100968,0.176720


Standardisation

In [101]:
scaler = StandardScaler()

In [102]:
# Fit and transform the selected columns
df_essentia_best_valence_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_valence_features_mean), columns=df_essentia_best_valence_features_mean.columns)

df_essentia_best_valence_features_mean_standardised = df_essentia_best_valence_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_valence_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_valence_features_mean_standardised.to_csv(get_deam_path('processed/features/standardised_essentia_best_valence_features_mean.csv'))

df_essentia_best_valence_features_mean_standardised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,lowlevel.zerocrossingrate.mean,rhythm.beats_loudness.mean,rhythm.onset_rate,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,2,-0.058571,0.380991,-0.069127,-0.384678,-0.303938,-0.369330,-0.452484,0.809781,-0.085112,...,0.231912,-0.169071,0.693755,-0.215917,0.174944,-0.382824,-0.398789,0.323740,-0.466431,-0.596685
1,3,1.479718,2.085477,3.025151,-1.355617,3.193846,-0.471575,-0.386129,0.202518,0.233522,...,-0.220673,-0.312868,0.297680,-0.477019,-0.368661,-0.434446,-0.398789,-0.487958,-0.563641,-0.386679
2,4,-0.208423,0.235235,0.521816,-0.348959,0.554801,0.552564,1.264373,0.275493,-0.282729,...,-0.220673,-0.292305,-0.231753,-0.477019,0.214373,-0.434446,-0.398789,-0.487958,-0.747808,-0.701512
3,5,-0.262338,-0.207226,-0.341279,-0.394848,-0.379636,-0.332708,1.199370,-0.827397,1.617190,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,0.004792,-0.398789,-0.439206,0.248861,-0.510704
4,7,2.154544,2.097370,2.747921,-1.381378,2.181744,1.365157,-1.793781,0.606843,-1.168573,...,-0.220673,-0.312868,-0.275872,0.206820,-0.368661,0.998061,1.904098,0.729588,-0.747808,-0.568093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,-0.321333,-0.139660,0.382933,-0.531866,0.001499,-0.174429,0.081083,0.775046,0.819838,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,-0.434446,-0.398789,-0.487958,-0.671897,-0.520047
1740,1997,-0.346532,-0.099817,0.297799,-0.359912,-0.183901,-0.921413,0.051395,0.428232,-0.546505,...,-0.220673,-0.312868,-0.275872,-0.477019,-0.368661,-0.240465,-0.398789,0.878506,0.998161,1.772008
1741,1998,-0.242062,0.036691,-0.052204,0.790683,0.057072,0.781015,0.025573,0.971682,0.513806,...,-0.220673,-0.065850,-0.275872,3.771523,-0.368661,2.682184,-0.398789,-0.487958,-0.444162,-0.701502
1742,1999,-0.098127,0.390945,-0.085379,-0.387172,-0.026040,0.520439,-1.326377,0.769227,0.185179,...,1.114688,-0.086435,4.781004,-0.427183,1.810241,0.044041,-0.398789,4.066921,0.043842,0.883837
